In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [3]:
from encoder4editing.models.psp import pSp


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


In [3]:
!pip install face_recognition 


In [4]:
!pip install torch torchvision opencv-python scikit-learn matplotlib
!git clone https://github.com/omertov/encoder4editing.git
!pip install -r encoder4editing/requirements.txt

  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires numpy<2.0.0,>=1.24.0, but you have numpy 2.0.2 which is incompatible.
facenet-pytorch 2.6.0 requires torch<2.3.0,>=2.2.0, but you have torch 2.5.1 which is incompatible.
facenet-pytorch 2.6.0 requires torchvision<0.18.0,>=0.17.0, but you have torchvision 0.20.1 which is incompatible.
fatal: destination path 'encoder4editing' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'encoder4editing/requirements.txt'


In [5]:
!pip install gdown --user


In [6]:
!pip install opencv-python torch


In [5]:
import torch

print("Torch version:", torch.__version__)
print("CUDA dispo :", torch.cuda.is_available())
print("Nom GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")


Torch version: 2.5.1
CUDA dispo : True
Nom GPU : NVIDIA GeForce GTX 1650 Ti with Max-Q Design


In [6]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.5.1
True
NVIDIA GeForce GTX 1650 Ti with Max-Q Design


In [6]:
from encoder4editing.models.psp import pSp
from encoder4editing.utils.common import tensor2im


In [2]:
# 📁 0. Imports
import os
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from encoder4editing.models.psp import pSp
from encoder4editing.utils.common import tensor2im
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 🧠 Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from types import SimpleNamespace

ckpt_path = 'encoder4editing/pretrained_models/e4e_ffhq_encode.pt'

def load_model():
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = SimpleNamespace(**opts)
    model = pSp(opts)       # ✅ Corrigé ici
    model = model.to(device)  # ✅ Envoyer sur le bon device
    model.eval()
    return model

model = load_model()


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_17584\4242676774.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Loading e4e over the pSp framework from checkpoint: encoder4editing/pretrained_models/e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

In [1]:
!pip install imbalanced-learn


In [3]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings

# Ignorer les avertissements sklearn/pytorch inutiles
warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Extraction des features à partir des frames
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=4):
    features_list, labels_list = [], []

    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Traitement de {len(frame_paths)} images pour '{label}' dans {folder_path}...")

    if len(frame_paths) == 0:
        print(f"⚠️ Aucune image trouvée dans {folder_path}")
        return features_list, labels_list

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    model.eval()

    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                img_tensor = transform(img)
                batch_images.append(img_tensor)
            except Exception as e:
                print(f"❌ Erreur chargement {path}: {e}")
                continue

        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)

        with torch.no_grad():
            batch_features = model(batch_tensor)

        for feat in batch_features:
            feature_vector = feat.detach().cpu().numpy().flatten()
            features_list.append(feature_vector)
            labels_list.append(1 if label == 'fake' else 0)

        del batch_tensor, batch_features
        torch.cuda.empty_cache()

    return features_list, labels_list


# ---------------------------------------------
# 🧠 Chargement du modèle e4e/pSp
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace

    print('🔍 Chargement du modèle pSp/e4e...')
    assert os.path.exists(ckpt_path), f"⚠️ Checkpoint introuvable : {ckpt_path}"

    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)

    model = pSp(opts)
    model.to(device)
    model.eval()
    print(f"📌 Modèle chargé sur : {device}")
    return model


# ---------------------------------------------
# ⚙️ Entraînement du classificateur + visualisation
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    print("📊 Taille dataset avant équilibrage :", np.bincount(y))

    # Étape 1 : Oversampling
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    print("✅ Taille après équilibrage :", np.bincount(y_res))

    # Étape 2 : Normalisation (float32 pour économiser la mémoire)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)
    del X_res  # libère mémoire

    # Étape 3 : Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res
    )
    del X_scaled, y_res  # libère mémoire

    # Étape 4 : RandomForest
    clf = RandomForestClassifier(
        n_estimators=300,
        class_weight='balanced_subsample',
        random_state=42,
        max_depth=18,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    # Étape 5 : Évaluation
    y_pred = clf.predict(X_test)
    acc_train = accuracy_score(y_train, clf.predict(X_train))
    acc_test = accuracy_score(y_test, y_pred)

    print(f"🎯 Accuracy (train): {acc_train:.4f}")
    print(f"🎯 Accuracy (test) : {acc_test:.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

    # Étape 6 : Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Real', 'Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Matrice de confusion')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    # Étape 7 : Courbe d’accuracy train/test
    plt.figure(figsize=(6, 4))
    plt.bar(['Train', 'Test'], [acc_train, acc_test], color=['#4CAF50', '#2196F3'])
    plt.title('📊 Accuracy du modèle (train vs test)')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "accuracy_train_test.png"))
    plt.close()

    # Étape 8 : Validation croisée légère (échantillon)
    print("📈 Calcul de la validation croisée légère...")
    scores = cross_val_score(clf, X_train[:800], y_train[:800], cv=3, scoring='accuracy', n_jobs=-1)
    print(f"📊 Moyenne validation croisée (échantillon réduit): {scores.mean():.4f}")

    plt.figure(figsize=(6, 4))
    plt.plot(range(1, len(scores)+1), scores, marker='o', linestyle='--', color='#FF9800')
    plt.title('📈 Validation croisée légère')
    plt.xlabel('Fold')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "cross_validation_curve.png"))
    plt.close()

    print("✅ Graphiques sauvegardés dans :", output_dir)
    return clf, scaler


# ---------------------------------------------
# 🚀 MAIN PIPELINE
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results'

    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    features_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    features_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    X = np.array(features_real + features_fake)
    y = np.array(labels_real + labels_fake)
    print(f"📊 Total features: {len(X)} (Real: {len(features_real)}, Fake: {len(features_fake)})")

    if len(X) == 0:
        raise ValueError('❌ Aucune feature extraite — vérifie tes dossiers de frames.')

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Modèle et scaler sauvegardés dans : {output_dir}")


🔍 Chargement du modèle pSp/e4e...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
📌 Modèle chargé sur : cuda
🔄 Traitement de 790 images pour 'real' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [09:31<00:00,  2.89s/it]


🔄 Traitement de 3975 images pour 'fake' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [46:51<00:00,  2.83s/it]    


📊 Total features: 4765 (Real: 790, Fake: 3975)
📊 Taille dataset avant équilibrage : [ 790 3975]
✅ Taille après équilibrage : [3975 3975]
🎯 Accuracy (train): 0.9995
🎯 Accuracy (test) : 0.9673
              precision    recall  f1-score   support

        Real       0.94      0.99      0.97       795
        Fake       0.99      0.94      0.97       795

    accuracy                           0.97      1590
   macro avg       0.97      0.97      0.97      1590
weighted avg       0.97      0.97      0.97      1590



C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:145: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:146: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:156: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:157: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "accuracy_train_test.png"))


📈 Calcul de la validation croisée légère...
📊 Moyenne validation croisée (échantillon réduit): 0.5975
✅ Graphiques sauvegardés dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:172: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_1052\2413290988.py:173: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "cross_validation_curve.png"))


✅ Modèle et scaler sauvegardés dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results


In [9]:
import os
import cv2
import torch
import numpy as np
import joblib
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms

# --- Charger le modèle e4e/pSp ---
from encoder4editing.models.psp import pSp
from argparse import Namespace

def load_model(ckpt_path, device='cuda'):
    print('🔍 Chargement du modèle e4e...')
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)
    model = pSp(opts).to(device).eval()
    return model

# --- Extraire les frames d’une vidéo ---
def extract_frames(video_path, output_dir, frame_interval=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1
    cap.release()
    print(f"🎞️ {saved} frames extraites dans {output_dir}")
    return sorted(glob(os.path.join(output_dir, "*.jpg")))

# --- Extraire les features avec e4e ---
def extract_features(frame_paths, model, device='cuda', batch_size=4):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    features = []
    model.eval()
    for i in tqdm(range(0, len(frame_paths), batch_size), desc="🔍 Extraction features"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []
        for p in batch_paths:
            img = Image.open(p).convert('RGB')
            batch_images.append(transform(img))
        batch_tensor = torch.stack(batch_images).to(device)
        with torch.no_grad():
            batch_features = model(batch_tensor)
        for feat in batch_features:
            features.append(feat.detach().cpu().numpy().flatten())
    return np.array(features)

# --- Prédiction vidéo ---
def predict_video(video_path, model_path, scaler_path, classifier_path, tmp_frames_dir, device='cuda'):
    model = load_model(model_path, device)
    scaler = joblib.load(scaler_path)
    clf = joblib.load(classifier_path)

    frame_paths = extract_frames(video_path, tmp_frames_dir, frame_interval=10)
    if len(frame_paths) == 0:
        raise ValueError("⚠️ Aucune frame extraite de la vidéo !")

    X = extract_features(frame_paths, model, device)
    X_scaled = scaler.transform(X)
    preds = clf.predict_proba(X_scaled)[:, 1]  # probabilité "fake"
    mean_pred = preds.mean()

    print(f"📈 Moyenne de probabilité Fake : {mean_pred:.3f}")
    if mean_pred > 0.5:
        print("🚨 La vidéo est probablement FAKE")
    else:
        print("✅ La vidéo semble REAL")
    return mean_pred

# --- MAIN ---
if __name__ == "__main__":
    video_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\fake\id0_id4_0004.mp4"
    model_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt"
    scaler_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results\scaler.pkl"
    classifier_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results\deepfake_classifier.pkl"
    tmp_frames_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\temp_frames"

    predict_video(video_path, model_path, scaler_path, classifier_path, tmp_frames_dir)


🔍 Chargement du modèle e4e...
Loading e4e over the pSp framework from checkpoint: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🎞️ 33 frames extraites dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\temp_frames


🔍 Extraction features: 100%|██████████| 13/13 [06:06<00:00, 28.17s/it]

📈 Moyenne de probabilité Fake : 0.516
🚨 La vidéo est probablement FAKE


In [4]:
import os
import cv2
import torch
import numpy as np
import joblib
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms

# --- Charger le modèle e4e/pSp ---
from encoder4editing.models.psp import pSp
from argparse import Namespace

def load_model(ckpt_path, device='cuda'):
    print('🔍 Chargement du modèle e4e...')
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)
    model = pSp(opts).to(device).eval()
    return model

# --- Extraire les frames d’une vidéo ---
def extract_frames(video_path, output_dir, frame_interval=10):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{saved:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved += 1
        count += 1
    cap.release()
    print(f"🎞️ {saved} frames extraites dans {output_dir}")
    return sorted(glob(os.path.join(output_dir, "*.jpg")))

# --- Extraire les features avec e4e ---
def extract_features(frame_paths, model, device='cuda', batch_size=4):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    features = []
    model.eval()
    for i in tqdm(range(0, len(frame_paths), batch_size), desc="🔍 Extraction features"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []
        for p in batch_paths:
            img = Image.open(p).convert('RGB')
            batch_images.append(transform(img))
        batch_tensor = torch.stack(batch_images).to(device)
        with torch.no_grad():
            batch_features = model(batch_tensor)
        for feat in batch_features:
            features.append(feat.detach().cpu().numpy().flatten())
    return np.array(features)

# --- Prédiction vidéo ---
def predict_video(video_path, model_path, scaler_path, classifier_path, tmp_frames_dir, device='cuda'):
    model = load_model(model_path, device)
    scaler = joblib.load(scaler_path)
    clf = joblib.load(classifier_path)

    frame_paths = extract_frames(video_path, tmp_frames_dir, frame_interval=10)
    if len(frame_paths) == 0:
        raise ValueError("⚠️ Aucune frame extraite de la vidéo !")

    X = extract_features(frame_paths, model, device)
    X_scaled = scaler.transform(X)
    preds = clf.predict_proba(X_scaled)[:, 1]  # probabilité "fake"
    mean_pred = preds.mean()

    print(f"📈 Moyenne de probabilité Fake : {mean_pred:.3f}")
    if mean_pred > 0.5:
        print("🚨 La vidéo est probablement FAKE")
    else:
        print("✅ La vidéo semble REAL")
    return mean_pred

# --- MAIN ---
if __name__ == "__main__":
    video_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\real\id0_0003.mp4"
    model_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt"
    scaler_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results\scaler.pkl"
    classifier_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results\deepfake_classifier.pkl"
    tmp_frames_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\temp_frames"

    predict_video(video_path, model_path, scaler_path, classifier_path, tmp_frames_dir)


🔍 Chargement du modèle e4e...
Loading e4e over the pSp framework from checkpoint: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🎞️ 53 frames extraites dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\temp_frames


🔍 Extraction features: 100%|██████████| 14/14 [06:07<00:00, 26.28s/it]

📈 Moyenne de probabilité Fake : 0.697
🚨 La vidéo est probablement FAKE


In [12]:
!pip install lightgbm


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------- ----------- 1.0/1.5 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 9.5 MB/s  0:00:00


In [1]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Extraction rapide des features
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=16):
    features_list, labels_list = [], []

    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Traitement de {len(frame_paths)} images pour '{label}' dans {folder_path}...")

    if len(frame_paths) == 0:
        print(f"⚠️ Aucune image trouvée dans {folder_path}")
        return features_list, labels_list

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    model.eval()

    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                batch_images.append(transform(img))
            except Exception as e:
                print(f"❌ Erreur chargement {path}: {e}")
                continue

        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)

        with torch.no_grad():
            batch_features = model(batch_tensor)

        # ⚡️ Réduction spatiale (moyenne globale pour réduire dimension)
        # Passe de 196,608 features → ~512-1024 selon le modèle
        batch_features = torch.mean(batch_features, dim=[2, 3])

        features_list.extend(batch_features.cpu().numpy())
        labels_list.extend([1 if label == 'fake' else 0] * len(batch_features))

        del batch_tensor, batch_features
        torch.cuda.empty_cache()

    return features_list, labels_list


# ---------------------------------------------
# 🧠 Chargement du modèle e4e/pSp
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace

    print('🔍 Chargement du modèle pSp/e4e...')
    assert os.path.exists(ckpt_path), f"⚠️ Checkpoint introuvable : {ckpt_path}"

    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)

    model = pSp(opts)
    model.to(device)
    model.eval()
    print(f"📌 Modèle chargé sur : {device}")
    return model


# ---------------------------------------------
# ⚙️ Entraînement LightGBM optimisé
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    print("📊 Taille dataset avant équilibrage :", np.bincount(y))

    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    print("✅ Taille après équilibrage :", np.bincount(y_res))

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)
    del X_res

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res
    )
    del X_scaled, y_res

    # ⚡️ LightGBM optimisé
    clf = LGBMClassifier(
        n_estimators=500,          # ↓ Moins d’arbres = plus rapide
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        force_col_wise=True,       # ⚡️ supprime l’overhead CPU
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )

    clf.fit(X_train, y_train)

    # ✅ Évaluation
    y_pred = clf.predict(X_test)
    acc_train = accuracy_score(y_train, clf.predict(X_train))
    acc_test = accuracy_score(y_test, y_pred)

    print(f"🎯 Accuracy (train): {acc_train:.4f}")
    print(f"🎯 Accuracy (test) : {acc_test:.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

    # 📊 Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Real', 'Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Matrice de confusion')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    # 📈 Accuracy bar
    plt.figure(figsize=(6, 4))
    plt.bar(['Train', 'Test'], [acc_train, acc_test], color=['#4CAF50', '#2196F3'])
    plt.title('📊 Accuracy du modèle (train vs test)')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "accuracy_train_test.png"))
    plt.close()

    # Validation croisée rapide
    print("📈 Validation croisée rapide...")
    scores = cross_val_score(clf, X_train[:500], y_train[:500], cv=3, scoring='accuracy', n_jobs=-1)
    print(f"📊 Moyenne validation croisée (sample): {scores.mean():.4f}")

    plt.figure(figsize=(6, 4))
    plt.plot(range(1, len(scores)+1), scores, marker='o', linestyle='--', color='#FF9800')
    plt.title('📈 Validation croisée rapide')
    plt.xlabel('Fold')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "cross_validation_curve.png"))
    plt.close()

    print("✅ Graphiques sauvegardés dans :", output_dir)
    return clf, scaler


# ---------------------------------------------
# 🚀 MAIN PIPELINE
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_fastss'

    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    features_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    features_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    X = np.array(features_real + features_fake)
    y = np.array(labels_real + labels_fake)
    print(f"📊 Total features: {len(X)} (Real: {len(features_real)}, Fake: {len(features_fake)})")

    if len(X) == 0:
        raise ValueError('❌ Aucune feature extraite — vérifie tes dossiers de frames.')

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Modèle et scaler sauvegardés dans : {output_dir}")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


🔍 Chargement du modèle pSp/e4e...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
📌 Modèle chargé sur : cuda
🔄 Traitement de 790 images pour 'real' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [04:43<00:00,  1.43s/it]


🔄 Traitement de 3975 images pour 'fake' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [23:07<00:00,  1.40s/it]  


...

c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:146: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:147: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))


🎯 Accuracy (train): 0.8615
🎯 Accuracy (test) : 0.7811
              precision    recall  f1-score   support

        Real       0.77      0.80      0.78       795
        Fake       0.79      0.76      0.78       795

    accuracy                           0.78      1590
   macro avg       0.78      0.78      0.78      1590
weighted avg       0.78      0.78      0.78      1590



C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:157: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:158: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "accuracy_train_test.png"))


📈 Validation croisée rapide...
📊 Moyenne validation croisée (sample): 0.5860
✅ Graphiques sauvegardés dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_fastss
✅ Modèle et scaler sauvegardés dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_fastss


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:173: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12412\3995857957.py:174: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "cross_validation_curve.png"))


In [1]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from skimage.metrics import structural_similarity as ssim

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Helper : convert tensor image to numpy [0,1]
# ---------------------------------------------
def tensor_to_numpy01(tensor):
    """
    Convert a torch tensor image (C,H,W) or (N,C,H,W) to numpy in range [0,1].
    Handles tensors in ranges [0,1] or [-1,1].
    """
    t = tensor.detach().cpu()
    # single image or batch
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")

    # If values look like [-1,1], convert to [0,1]
    if arr.min() < -0.1 or arr.max() > 1.1:
        # clamp then map [-1,1] -> [0,1]
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)

    return arr

# ---------------------------------------------
# 🔧 Compute reconstruction errors (robust)
# ---------------------------------------------
def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    """
    Encode -> Reconstruct (via model forward) -> Compute MSE and (1-SSIM) per image.
    Uses model(image_tensor, randomize_noise=False, resize=False) to reconstruct.
    """
    model.eval()
    with torch.no_grad():
        try:
            # Try to get reconstructed images using the model's forward
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            # Some older wrappers may have different signature - try with only image
            out = model(image_tensor)

        # out might be a tensor, or tuple/list where first element is images
        if isinstance(out, (tuple, list)):
            reconstructed = out[0]
        else:
            reconstructed = out

    # Ensure shapes align
    if reconstructed.shape != image_tensor.shape:
        # attempt to resize or raise informative error
        # Some implementations return images in [0,1] or [-1,1] and may also return different size.
        # We will try to interpolate to input size if spatial dims mismatch.
        if reconstructed.dim() == 4 and image_tensor.dim() == 4:
            if reconstructed.shape[2:] != image_tensor.shape[2:]:
                reconstructed = torch.nn.functional.interpolate(
                    reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
                )
        else:
            raise RuntimeError(f"Shape mismatch between input {image_tensor.shape} and reconstructed {reconstructed.shape}")

    # Convert to numpy [0,1]
    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    # Compute MSE per image
    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1,2,3))

    # Compute SSIM per image (skimage expects H,W,C, values in [0,1])
    ssim_scores = []
    for i in range(orig_np.shape[0]):
        try:
            s = ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0)
        except TypeError:
            # fallback for older skimage versions using multichannel argument
            s = ssim(orig_np[i], rec_np[i], multichannel=True, data_range=1.0)
        ssim_scores.append(s)

    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors

# ---------------------------------------------
# 🔧 Process frames in batches -> return error features
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=4):
    """Calcule les erreurs de reconstruction pour chaque image."""
    errors_list, labels_list = [], []

    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Traitement de {len(frame_paths)} images pour '{label}' dans {folder_path}...")

    if len(frame_paths) == 0:
        print(f"⚠️ Aucune image trouvée dans {folder_path}")
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),  # range [0,1]
    ])

    model.eval()

    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                img_tensor = transform(img)
                batch_images.append(img_tensor)
            except Exception as e:
                print(f"❌ Erreur chargement {path}: {e}")
                continue

        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)

        try:
            errors = compute_reconstruction_errors(model, batch_tensor, device=device)
            errors_list.append(errors)
            labels_list += [1 if label == 'fake' else 0] * len(errors)
        except Exception as e:
            print(f"❌ Erreur pendant reconstruction/mesure sur un batch: {e}")
            # pour robustesse on skip ce batch
            continue
        finally:
            del batch_tensor
            torch.cuda.empty_cache()

    if len(errors_list) == 0:
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)

    errors_array = np.vstack(errors_list)
    return errors_array.astype(np.float32), np.array(labels_list, dtype=np.int32)

# ---------------------------------------------
# 🧠 Chargement du modèle e4e/pSp
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace

    print('🔍 Chargement du modèle pSp/e4e...')
    assert os.path.exists(ckpt_path), f"⚠️ Checkpoint introuvable : {ckpt_path}"

    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)

    model = pSp(opts)
    model.to(device)
    model.eval()
    print(f"📌 Modèle chargé sur : {device}")
    return model

# ---------------------------------------------
# ⚙️ Entraînement du classificateur + visualisation
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    if len(X) == 0:
        raise ValueError("Aucun échantillon fourni au classifieur.")

    print("📊 Taille dataset avant équilibrage :", np.bincount(y))

    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    print("✅ Taille après équilibrage :", np.bincount(y_res))

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res
    )

    clf = RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        max_depth=10,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc_train = accuracy_score(y_train, clf.predict(X_train))
    acc_test = accuracy_score(y_test, y_pred)

    print(f"🎯 Accuracy (train): {acc_train:.4f}")
    print(f"🎯 Accuracy (test) : {acc_test:.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Real', 'Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Matrice de confusion')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    return clf, scaler

# ---------------------------------------------
# 🚀 MAIN PIPELINE
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan'

    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    # Ajuste batch_size selon ta VRAM si besoin
    errors_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    errors_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    if errors_real.size == 0 and errors_fake.size == 0:
        raise RuntimeError("Aucune erreur calculée — vérifie les chemins et le checkpoint du modèle.")

    X = np.vstack([errors_real, errors_fake]) if errors_real.size and errors_fake.size else (errors_real if errors_fake.size == 0 else errors_fake)
    y = np.concatenate([labels_real, labels_fake]) if (labels_real.size and labels_fake.size) else (labels_real if labels_fake.size == 0 else labels_fake)
    print(f"📊 Total samples: {len(X)} (Real: {len(errors_real)}, Fake: {len(errors_fake)})")

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Modèle et scaler sauvegardés dans : {output_dir}")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


🔍 Chargement du modèle pSp/e4e...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
📌 Modèle chargé sur : cuda
🔄 Traitement de 790 images pour 'real' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [04:53<00:00,  1.48s/it]


🔄 Traitement de 3975 images pour 'fake' dans C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [24:08<00:00,  1.46s/it]


📊 Total samples: 4765 (Real: 790, Fake: 3975)
📊 Taille dataset avant équilibrage : [ 790 3975]
✅ Taille après équilibrage : [3975 3975]
🎯 Accuracy (train): 0.7975
🎯 Accuracy (test) : 0.7195
              precision    recall  f1-score   support

        Real       0.82      0.56      0.67       795
        Fake       0.67      0.87      0.76       795

    accuracy                           0.72      1590
   macro avg       0.74      0.72      0.71      1590
weighted avg       0.74      0.72      0.71      1590



C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_19292\1633671959.py:231: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_19292\1633671959.py:232: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))


✅ Modèle et scaler sauvegardés dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan


In [1]:
# ==============================================
# 🚀 Pipeline complet : GAN Inversion + DeepFake Detection
# ==============================================
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from skimage.metrics import structural_similarity as ssim

warnings.filterwarnings("ignore", category=FutureWarning)


# ---------------------------------------------
# 🔧 Convert torch tensor to numpy [0,1]
# ---------------------------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")

    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1, 1)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr


# ---------------------------------------------
# 🔧 Compute reconstruction errors
# ---------------------------------------------
def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)

        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    # Resize if necessary
    if reconstructed.shape != image_tensor.shape:
        if reconstructed.dim() == 4 and image_tensor.dim() == 4:
            if reconstructed.shape[2:] != image_tensor.shape[2:]:
                reconstructed = torch.nn.functional.interpolate(
                    reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
                )
        else:
            raise RuntimeError(f"Shape mismatch {image_tensor.shape} vs {reconstructed.shape}")

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1, 2, 3))

    ssim_scores = []
    for i in range(orig_np.shape[0]):
        try:
            s = ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0)
        except TypeError:
            s = ssim(orig_np[i], rec_np[i], multichannel=True, data_range=1.0)
        ssim_scores.append(s)

    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors


# ---------------------------------------------
# 🔧 Process folder images in batches
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=4):
    errors_list, labels_list = [], []

    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Processing {len(frame_paths)} images for '{label}' in {folder_path}")

    if len(frame_paths) == 0:
        print(f"⚠️ No images found in {folder_path}")
        return np.zeros((0, 2), dtype=np.float32), np.array([], dtype=np.int32)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i + batch_size]
        batch_images = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                img_tensor = transform(img)
                batch_images.append(img_tensor)
            except Exception as e:
                print(f"❌ Failed loading {path}: {e}")
                continue

        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)

        try:
            errors = compute_reconstruction_errors(model, batch_tensor, device=device)
            errors_list.append(errors)
            labels_list += [1 if label == 'fake' else 0] * len(errors)
        except Exception as e:
            print(f"❌ Error in batch: {e}")
            continue
        finally:
            del batch_tensor
            torch.cuda.empty_cache()

    if len(errors_list) == 0:
        return np.zeros((0, 2), dtype=np.float32), np.array([], dtype=np.int32)

    errors_array = np.vstack(errors_list)
    return errors_array.astype(np.float32), np.array(labels_list, dtype=np.int32)


# ---------------------------------------------
# 🧠 Load e4e/pSp model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace

    print('🔍 Loading pSp/e4e model...')
    assert os.path.exists(ckpt_path), f"⚠️ Checkpoint not found: {ckpt_path}"

    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)

    model = pSp(opts)
    model.to(device)
    model.eval()
    print(f"📌 Model loaded on {device}")
    return model


# ---------------------------------------------
# ⚙️ Train classifier and visualize
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    if len(X) == 0:
        raise ValueError("No samples provided.")

    print("📊 Dataset size before balancing:", np.bincount(y))

    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)
    print("✅ Dataset size after balancing:", np.bincount(y_res))

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res
    )

    clf = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    print(f"🎯 Accuracy (train): {accuracy_score(y_train, clf.predict(X_train)):.4f}")
    print(f"🎯 Accuracy (test): {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Real', 'Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    return clf, scaler


# ---------------------------------------------
# 🚀 MAIN PIPELINE
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan'
    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    # Batch size adjustable
    errors_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    errors_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    if errors_real.size == 0 and errors_fake.size == 0:
        raise RuntimeError("No errors computed — check paths and model checkpoint.")

    X = np.vstack([errors_real, errors_fake]) if errors_real.size and errors_fake.size else (errors_real if errors_fake.size == 0 else errors_fake)
    y = np.concatenate([labels_real, labels_fake]) if (labels_real.size and labels_fake.size) else (labels_real if labels_fake.size == 0 else labels_fake)

    print(f"📊 Total samples: {len(X)} (Real: {len(errors_real)}, Fake: {len(errors_fake)})")

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Classifier and scaler saved to: {output_dir}")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


🔍 Loading pSp/e4e model...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
📌 Model loaded on cuda
🔄 Processing 790 images for 'real' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real


Processing real: 100%|██████████| 198/198 [04:49<00:00,  1.46s/it]


🔄 Processing 3975 images for 'fake' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake


Processing fake: 100%|██████████| 994/994 [22:45<00:00,  1.37s/it]


📊 Total samples: 4765 (Real: 790, Fake: 3975)
📊 Dataset size before balancing: [ 790 3975]
✅ Dataset size after balancing: [3975 3975]
🎯 Accuracy (train): 0.7984
🎯 Accuracy (test): 0.7208
              precision    recall  f1-score   support

        Real       0.82      0.57      0.67       795
        Fake       0.67      0.88      0.76       795

    accuracy                           0.72      1590
   macro avg       0.74      0.72      0.71      1590
weighted avg       0.74      0.72      0.71      1590



C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12832\3221495025.py:202: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12832\3221495025.py:203: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))


✅ Classifier and scaler saved to: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan


In [4]:
import os
import cv2
import torch
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
from skimage.metrics import structural_similarity as ssim
import joblib
from argparse import Namespace

# ------------------------
# Config
# ------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_ckpt = r"encoder4editing\pretrained_models\e4e_ffhq_encode.pt"
classifier_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan\deepfake_classifier.pkl"
scaler_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan\scaler.pkl"
video_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\real\id0_0000.mp4"
frame_step = 5  # tester 1 frame toutes les 5 pour accélérer

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# ------------------------
# Fonctions utilitaires
# ------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    else:
        arr = t.permute(0, 2, 3, 1).numpy()
    arr = np.clip((arr + 1)/2.0, 0.0, 1.0) if arr.min() < 0 else np.clip(arr, 0.0, 1.0)
    return arr

def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = ckpt['opts']
    opts['checkpoint_path'] = ckpt_path
    opts = Namespace(**opts)
    model = pSp(opts)
    model.to(device).eval()
    return model

def compute_errors(img, model):
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        try:
            out = model(img_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(img_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != img_tensor.shape:
        reconstructed = F.interpolate(reconstructed, size=img_tensor.shape[2:], mode='bilinear', align_corners=False)

    orig_np = tensor_to_numpy01(img_tensor)[0]
    rec_np = tensor_to_numpy01(reconstructed)[0]

    mse_loss = np.mean((orig_np - rec_np)**2)
    try:
        s = ssim(orig_np, rec_np, channel_axis=2, data_range=1.0)
    except TypeError:
        s = ssim(orig_np, rec_np, multichannel=True, data_range=1.0)

    return np.array([[mse_loss, 1 - s]], dtype=np.float32)

def predict_frame(img, model, classifier, scaler):
    errors = compute_errors(img, model)
    errors_scaled = scaler.transform(errors)
    pred = classifier.predict(errors_scaled)
    return pred[0]

# ------------------------
# Charger modèle et classifieur
# ------------------------
model = load_model(model_ckpt, device=device)
classifier = joblib.load(classifier_path)
scaler = joblib.load(scaler_path)

# ------------------------
# Tester vidéo
# ------------------------
cap = cv2.VideoCapture(video_path)
frame_preds = []
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % frame_step == 0:  # échantillonnage
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        try:
            pred = predict_frame(img, model, classifier, scaler)
            frame_preds.append(pred)
        except Exception as e:
            print(f"❌ Frame error: {e}")
    frame_idx += 1

cap.release()

if frame_preds:
    fake_ratio = np.mean(frame_preds)
    video_label = "Fake" if fake_ratio > 0.5 else "Real"
    print(f"🎬 {os.path.basename(video_path)} --> {video_label} (fake_ratio={fake_ratio:.2f})")
else:
    print("❌ Aucun frame traité, vérifie la vidéo")


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🎬 id0_0000.mp4 --> Fake (fake_ratio=0.70)


In [5]:
!pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
    --------------------------------------- 1.6/124.9 MB 9.3 MB/s eta 0:00:14
   - -------------------------------------- 4.2/124.9 MB 10.5 MB/s eta 0:00:12
   - -------------------------------------- 6.0/124.9 MB 10.2 MB/s eta 0:00:12
   -- ------------------------------------- 8.7/124.9 MB 10.7 MB/s eta 0:00:11
   --- ------------------------------------ 10.7/124.9 MB 10.6 MB/s eta 0:00:11
   ---- ----------------------------------- 13.4/124.9 MB 10.9 MB/s eta 0:00:11
   ---- ----------------------------------- 14.7/124.9 MB 11.0 MB/s eta 0:00:11
   ----- ---------------------------------- 16.5/124.9 MB 10.1 MB/s eta 0:00:11
   ------ --------------------------------- 19.1/124.9 MB 10.3 MB/s eta 0:00:11
   ------ --------------------------------- 21.8/124.9 MB 10.5 MB/s eta 0:00:10
   ------- -------------------------------- 24.4/124.9 MB 10.6 MB/s eta 0:00:10
   -------- ------------------------------- 26.7/124.9

In [ ]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from skimage.metrics import structural_similarity as ssim
import xgboost as xgb  # ✅ Utilisation de XGBoost

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Helper : convert tensor image to numpy [0,1]
# ---------------------------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

# ---------------------------------------------
# 🔧 Compute reconstruction errors
# ---------------------------------------------
def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != image_tensor.shape:
        if reconstructed.dim() == 4 and image_tensor.dim() == 4:
            if reconstructed.shape[2:] != image_tensor.shape[2:]:
                reconstructed = torch.nn.functional.interpolate(
                    reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
                )
        else:
            raise RuntimeError(f"Shape mismatch {image_tensor.shape} vs {reconstructed.shape}")

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1,2,3))
    ssim_scores = []
    for i in range(orig_np.shape[0]):
        try:
            s = ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0)
        except TypeError:
            s = ssim(orig_np[i], rec_np[i], multichannel=True, data_range=1.0)
        ssim_scores.append(s)

    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors

# ---------------------------------------------
# 🔧 Process frames from folder
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=4):
    errors_list, labels_list = [], []
    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Processing {len(frame_paths)} images for '{label}' in {folder_path}...")
    if len(frame_paths) == 0:
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    model.eval()
    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                batch_images.append(transform(img))
            except Exception as e:
                print(f"❌ Failed loading {path}: {e}")
        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)
        try:
            errors = compute_reconstruction_errors(model, batch_tensor, device=device)
            errors_list.append(errors)
            labels_list += [1 if label=='fake' else 0]*len(errors)
        except Exception as e:
            print(f"❌ Error in batch: {e}")
        finally:
            del batch_tensor
            torch.cuda.empty_cache()

    if len(errors_list) == 0:
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)
    errors_array = np.vstack(errors_list)
    return errors_array.astype(np.float32), np.array(labels_list, dtype=np.int32)

# ---------------------------------------------
# 🧠 Load e4e/pSp model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    model.eval()
    return model

# ---------------------------------------------
# ⚙️ Train classifier (XGBoost)
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, stratify=y_res, random_state=42
    )

    clf = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    print(f"🎯 Accuracy (train): {accuracy_score(y_train, clf.predict(X_train)):.4f}")
    print(f"🎯 Accuracy (test) : {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real','Fake']))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=['Real','Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    return clf, scaler

# ---------------------------------------------
# 🚀 MAIN
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing','pretrained_models','e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust'
    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    errors_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    errors_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    if errors_real.size==0 and errors_fake.size==0:
        raise RuntimeError("No errors computed. Check paths or model checkpoint.")

    X = np.vstack([errors_real, errors_fake])
    y = np.concatenate([labels_real, labels_fake])
    print(f"📊 Total samples: {len(X)} (Real: {len(errors_real)}, Fake: {len(errors_fake)})")

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier_xgb.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Classifier and scaler saved in {output_dir}")


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🔄 Processing 790 images for 'real' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [06:56<00:00,  2.10s/it]


🔄 Processing 3975 images for 'fake' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [22:15<00:00,  1.34s/it]


📊 Total samples: 4765 (Real: 790, Fake: 3975)
🎯 Accuracy (train): 0.7053
🎯 Accuracy (test) : 0.6264
              precision    recall  f1-score   support

        Real       0.63      0.63      0.63       795
        Fake       0.63      0.63      0.63       795

    accuracy                           0.63      1590
   macro avg       0.63      0.63      0.63      1590
weighted avg       0.63      0.63      0.63      1590



C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12832\2818429873.py:178: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_12832\2818429873.py:179: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))


✅ Classifier and scaler saved in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust


In [2]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from skimage.metrics import structural_similarity as ssim
import xgboost as xgb  # ✅ Utilisation de XGBoost

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Helper : convert tensor image to numpy [0,1]
# ---------------------------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

# ---------------------------------------------
# 🔧 Compute reconstruction errors
# ---------------------------------------------
def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != image_tensor.shape:
        if reconstructed.dim() == 4 and image_tensor.dim() == 4:
            if reconstructed.shape[2:] != image_tensor.shape[2:]:
                reconstructed = torch.nn.functional.interpolate(
                    reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
                )
        else:
            raise RuntimeError(f"Shape mismatch {image_tensor.shape} vs {reconstructed.shape}")

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1,2,3))
    ssim_scores = []
    for i in range(orig_np.shape[0]):
        try:
            s = ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0)
        except TypeError:
            s = ssim(orig_np[i], rec_np[i], multichannel=True, data_range=1.0)
        ssim_scores.append(s)

    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors

# ---------------------------------------------
# 🔧 Process frames from folder
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, label, device='cuda', batch_size=4):
    errors_list, labels_list = [], []
    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Processing {len(frame_paths)} images for '{label}' in {folder_path}...")
    if len(frame_paths) == 0:
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    model.eval()
    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                batch_images.append(transform(img))
            except Exception as e:
                print(f"❌ Failed loading {path}: {e}")
        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)
        try:
            errors = compute_reconstruction_errors(model, batch_tensor, device=device)
            errors_list.append(errors)
            labels_list += [1 if label=='fake' else 0]*len(errors)
        except Exception as e:
            print(f"❌ Error in batch: {e}")
        finally:
            del batch_tensor
            torch.cuda.empty_cache()

    if len(errors_list) == 0:
        return np.zeros((0,2), dtype=np.float32), np.array([], dtype=np.int32)
    errors_array = np.vstack(errors_list)
    return errors_array.astype(np.float32), np.array(labels_list, dtype=np.int32)

# ---------------------------------------------
# 🧠 Load e4e/pSp model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    model.eval()
    return model

# ---------------------------------------------
# ⚙️ Train classifier (XGBoost)
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, stratify=y_res, random_state=42
    )

    clf = xgb.XGBClassifier(
        n_estimators=600,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    print(f"🎯 Accuracy (train): {accuracy_score(y_train, clf.predict(X_train)):.4f}")
    print(f"🎯 Accuracy (test) : {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real','Fake']))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=['Real','Fake'])
    disp.plot(cmap='Blues')
    plt.title('🧩 Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    return clf, scaler

# ---------------------------------------------
# 🚀 MAIN
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing','pretrained_models','e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6'
    os.makedirs(output_dir, exist_ok=True)

    model = load_model(ckpt_path, device=device)

    errors_real, labels_real = process_frames_from_folder(path_real, model, 'real', device=device, batch_size=4)
    errors_fake, labels_fake = process_frames_from_folder(path_fake, model, 'fake', device=device, batch_size=4)

    if errors_real.size==0 and errors_fake.size==0:
        raise RuntimeError("No errors computed. Check paths or model checkpoint.")

    X = np.vstack([errors_real, errors_fake])
    y = np.concatenate([labels_real, labels_fake])
    print(f"📊 Total samples: {len(X)} (Real: {len(errors_real)}, Fake: {len(errors_fake)})")

    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier_xgb.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Classifier and scaler saved in {output_dir}")


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🔄 Processing 790 images for 'real' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [05:01<00:00,  1.52s/it]


🔄 Processing 3975 images for 'fake' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [23:32<00:00,  1.42s/it]  


📊 Total samples: 4765 (Real: 790, Fake: 3975)
🎯 Accuracy (train): 0.7066
🎯 Accuracy (test) : 0.6314
              precision    recall  f1-score   support

        Real       0.63      0.63      0.63       795
        Fake       0.63      0.64      0.63       795

    accuracy                           0.63      1590
   macro avg       0.63      0.63      0.63      1590
weighted avg       0.63      0.63      0.63      1590

✅ Classifier and scaler saved in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_8780\80568076.py:178: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_8780\80568076.py:179: UserWarning: Glyph 129513 (\N{JIGSAW PUZZLE PIECE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))


In [6]:
import os
import cv2
import numpy as np
import joblib
import torch
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
from encoder4editing.models.psp import pSp
from argparse import Namespace
import matplotlib.pyplot as plt

# =============================================
# 🔧 Fonctions utilitaires
# =============================================

def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != image_tensor.shape:
        reconstructed = torch.nn.functional.interpolate(
            reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
        )

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3: orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3: rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1,2,3))
    ssim_scores = [ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0) for i in range(orig_np.shape[0])]
    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors

def load_model(ckpt_path, device='cuda'):
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    model.eval()
    return model

# =============================================
# 🎥 Test d'une vidéo
# =============================================

def test_video(video_path, model, classifier, scaler, device='cuda', output_dir='./results_test', frame_skip=10):
    os.makedirs(output_dir, exist_ok=True)
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎞️ Total frames: {total_frames}")

    frame_idx = 0
    all_errors = []
    predictions = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_skip == 0:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(img)
            tensor_img = transform(pil_img).unsqueeze(0).to(device)

            try:
                errors = compute_reconstruction_errors(model, tensor_img, device=device)
                scaled = scaler.transform(errors.astype(np.float32))
                prob = classifier.predict_proba(scaled)[0, 1]  # probabilité de "Fake"
                predictions.append(prob)
                all_errors.append(errors[0])
            except Exception as e:
                print(f"⚠️ Frame {frame_idx}: {e}")

        frame_idx += 1

    cap.release()

    # --------------------------------------------
    # 📊 Résumé et visualisation
    # --------------------------------------------
    all_errors = np.array(all_errors)
    predictions = np.array(predictions)
    mean_fake_prob = predictions.mean()

    print(f"\n📈 Moyenne probabilité 'Fake' : {mean_fake_prob:.3f}")
    verdict = "🟥 FAKE" if mean_fake_prob > 0.6 else "🟩 REAL"
    print(f"✅ Résultat final pour la vidéo : {verdict}")

    # Sauvegarde du graphique
    plt.figure(figsize=(8,4))
    plt.plot(predictions, label='Probabilité Fake par frame', color='red')
    plt.axhline(0.5, color='gray', linestyle='--')
    plt.title(f"Détection GAN inversion — {verdict}")
    plt.xlabel("Frames (échantillons)")
    plt.ylabel("Probabilité Fake")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"predictions_{os.path.basename(video_path)}.png"))
    plt.close()

    return verdict, mean_fake_prob, predictions

# =============================================
# 🚀 MAIN
# =============================================
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ⚙️ Chemins à adapter
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    classifier_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6\deepfake_classifier_xgb.pkl'
    scaler_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6\scaler.pkl'

    # 🎥 ✅ Vidéo à tester
    video_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\real\id0_0003.mp4'

    # 📁 Dossier de sortie
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_test_video'

    print("📦 Chargement du modèle e4e/pSp ...")
    model = load_model(ckpt_path, device=device)

    print("📦 Chargement du classifieur et du scaler ...")
    classifier = joblib.load(classifier_path)
    scaler = joblib.load(scaler_path)

    # 🚀 Test de la vidéo
    verdict, score, preds = test_video(video_path, model, classifier, scaler, device=device, output_dir=output_dir)

    print(f"\n✅ Verdict final : {verdict} (score moyen = {score:.3f})")


📦 Chargement du modèle e4e/pSp ...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
📦 Chargement du classifieur et du scaler ...
🎞️ Total frames: 529

📈 Moyenne probabilité 'Fake' : 0.503
✅ Résultat final pour la vidéo : 🟩 REAL

✅ Verdict final : 🟩 REAL (score moyen = 0.503)


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_8780\3741065254.py:132: UserWarning: Glyph 129001 (\N{LARGE GREEN SQUARE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_8780\3741065254.py:133: UserWarning: Glyph 129001 (\N{LARGE GREEN SQUARE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, f"predictions_{os.path.basename(video_path)}.png"))


In [1]:
import os
import cv2
import numpy as np
import joblib
import torch
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
from encoder4editing.models.psp import pSp
from argparse import Namespace
import matplotlib.pyplot as plt

# =============================================
# 🔧 Fonctions utilitaires
# =============================================

def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

def compute_reconstruction_errors(model, image_tensor, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != image_tensor.shape:
        reconstructed = torch.nn.functional.interpolate(
            reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
        )

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3: orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3: rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1,2,3))
    ssim_scores = [ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0) for i in range(orig_np.shape[0])]
    errors = np.stack([mse_loss, 1.0 - np.array(ssim_scores)], axis=1)
    return errors

def load_model(ckpt_path, device='cuda'):
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    model.eval()
    return model

# =============================================
# 🎥 Test d'une vidéo
# =============================================

def test_video(video_path, model, classifier, scaler, device='cuda', output_dir='./results_test', frame_skip=10):
    os.makedirs(output_dir, exist_ok=True)
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"🎞️ Total frames: {total_frames}")

    frame_idx = 0
    all_errors = []
    predictions = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_skip == 0:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(img)
            tensor_img = transform(pil_img).unsqueeze(0).to(device)

            try:
                errors = compute_reconstruction_errors(model, tensor_img, device=device)
                scaled = scaler.transform(errors.astype(np.float32))
                prob = classifier.predict_proba(scaled)[0, 1]  # probabilité de "Fake"
                predictions.append(prob)
                all_errors.append(errors[0])
            except Exception as e:
                print(f"⚠️ Frame {frame_idx}: {e}")

        frame_idx += 1

    cap.release()

    # --------------------------------------------
    # 📊 Résumé et visualisation
    # --------------------------------------------
    all_errors = np.array(all_errors)
    predictions = np.array(predictions)
    mean_fake_prob = predictions.mean()

    print(f"\n📈 Moyenne probabilité 'Fake' : {mean_fake_prob:.3f}")
    verdict = "🟥 FAKE" if mean_fake_prob > 0.5 else "🟩 REAL"
    print(f"✅ Résultat final pour la vidéo : {verdict}")

    # Sauvegarde du graphique
    plt.figure(figsize=(8,4))
    plt.plot(predictions, label='Probabilité Fake par frame', color='red')
    plt.axhline(0.5, color='gray', linestyle='--')
    plt.title(f"Détection GAN inversion — {verdict}")
    plt.xlabel("Frames (échantillons)")
    plt.ylabel("Probabilité Fake")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"predictions_{os.path.basename(video_path)}.png"))
    plt.close()

    return verdict, mean_fake_prob, predictions

# =============================================
# 🚀 MAIN
# =============================================
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # ⚙️ Chemins à adapter
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    classifier_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6\deepfake_classifier_xgb.pkl'
    scaler_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust6\scaler.pkl'

    # 🎥 ✅ Vidéo à tester
    video_path = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\fake\id0_id17_0001.mp4'

    # 📁 Dossier de sortie
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_test_video'

    print("📦 Chargement du modèle e4e/pSp ...")
    model = load_model(ckpt_path, device=device)

    print("📦 Chargement du classifieur et du scaler ...")
    classifier = joblib.load(classifier_path)
    scaler = joblib.load(scaler_path)

    # 🚀 Test de la vidéo
    verdict, score, preds = test_video(video_path, model, classifier, scaler, device=device, output_dir=output_dir)

    print(f"\n✅ Verdict final : {verdict} (score moyen = {score:.3f})")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


📦 Chargement du modèle e4e/pSp ...


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_24692\687490820.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

📦 Chargement du classifieur et du scaler ...
🎞️ Total frames: 303

📈 Moyenne probabilité 'Fake' : 0.594
✅ Résultat final pour la vidéo : 🟥 FAKE

✅ Verdict final : 🟥 FAKE (score moyen = 0.594)


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_24692\687490820.py:132: UserWarning: Glyph 128997 (\N{LARGE RED SQUARE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_24692\687490820.py:133: UserWarning: Glyph 128997 (\N{LARGE RED SQUARE}) missing from font(s) DejaVu Sans.
  plt.savefig(os.path.join(output_dir, f"predictions_{os.path.basename(video_path)}.png"))


In [9]:
!pip install lpips


In [ ]:
import os
import numpy as np
import joblib
import torch
from glob import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import warnings
from skimage.metrics import structural_similarity as ssim
import torch.nn.functional as F
import xgboost as xgb
import lpips  # ✅ pip install lpips

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------------------------------------
# 🔧 Convert tensor image to numpy [0,1]
# ---------------------------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

# ---------------------------------------------
# 🔧 Compute reconstruction + perceptual errors
# ---------------------------------------------
def compute_reconstruction_errors(model, image_tensor, lpips_fn, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    # Ajuste les dimensions si nécessaire
    if reconstructed.shape != image_tensor.shape:
        reconstructed = torch.nn.functional.interpolate(
            reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
        )

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    # 🔹 MSE & SSIM
    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1, 2, 3))
    ssim_scores = [ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0) for i in range(orig_np.shape[0])]

    # 🔹 PSNR
    psnr_vals = [20 * np.log10(1.0 / np.sqrt(m + 1e-8)) for m in mse_loss]

    # 🔹 LPIPS (perceptual similarity)
    with torch.no_grad():
        lpips_vals = lpips_fn(image_tensor, reconstructed).detach().cpu().numpy().flatten()

    # 🔹 Cosine similarity
    cosine_vals = [
        F.cosine_similarity(image_tensor[i].flatten(), reconstructed[i].flatten(), dim=0).item()
        for i in range(image_tensor.shape[0])
    ]

    # ✅ Final feature vector
    errors = np.stack([
        mse_loss,                 # 1
        1.0 - np.array(ssim_scores),  # 2
        np.array(psnr_vals),      # 3
        np.array(lpips_vals),     # 4
        1.0 - np.array(cosine_vals)  # 5
    ], axis=1)
    return errors

# ---------------------------------------------
# 🔧 Process frames from folder
# ---------------------------------------------
def process_frames_from_folder(folder_path, model, lpips_fn, label, device='cuda', batch_size=4):
    errors_list, labels_list = [], []
    exts = ['jpg', 'jpeg', 'png']
    frame_paths = sorted(sum([
        glob(os.path.join(folder_path, '**', f'*.{ext}'), recursive=True)
        for ext in exts
    ], []))

    print(f"🔄 Processing {len(frame_paths)} images for '{label}' in {folder_path}...")
    if len(frame_paths) == 0:
        return np.zeros((0, 5), dtype=np.float32), np.array([], dtype=np.int32)

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

    for i in tqdm(range(0, len(frame_paths), batch_size), desc=f"Processing {label}"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                batch_images.append(transform(img))
            except Exception as e:
                print(f"❌ Failed loading {path}: {e}")
        if not batch_images:
            continue

        batch_tensor = torch.stack(batch_images).to(device)
        try:
            errors = compute_reconstruction_errors(model, batch_tensor, lpips_fn, device=device)
            errors_list.append(errors)
            labels_list += [1 if label == 'fake' else 0] * len(errors)
        except Exception as e:
            print(f"❌ Error in batch: {e}")
        finally:
            del batch_tensor
            torch.cuda.empty_cache()

    if len(errors_list) == 0:
        return np.zeros((0, 5), dtype=np.float32), np.array([], dtype=np.int32)
    errors_array = np.vstack(errors_list)
    return errors_array.astype(np.float32), np.array(labels_list, dtype=np.int32)

# ---------------------------------------------
# 🧠 Load e4e/pSp model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace
    assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    model.eval()
    return model

# ---------------------------------------------
# ⚙️ Train classifier (Optimized XGBoost)
# ---------------------------------------------
def train_balanced_classifier(X, y, output_dir):
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(X, y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_res).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_res, test_size=0.2, stratify=y_res, random_state=42
    )

    clf = xgb.XGBClassifier(
        n_estimators=1200,
        max_depth=8,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        gamma=0.1,
        reg_lambda=1.2,
        reg_alpha=0.5,
        min_child_weight=3,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    print(f"🎯 Accuracy (train): {accuracy_score(y_train, clf.predict(X_train)):.4f}")
    print(f"🎯 Accuracy (test) : {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=['Real', 'Fake'])
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    return clf, scaler

# ---------------------------------------------
# 🚀 MAIN
# ---------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
    path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust_optimized'
    os.makedirs(output_dir, exist_ok=True)

    print("🔹 Loading e4e model...")
    model = load_model(ckpt_path, device=device)

    print("🔹 Initializing LPIPS metric...")
    lpips_fn = lpips.LPIPS(net='alex').to(device)

    # 🔸 Extract features
    errors_real, labels_real = process_frames_from_folder(path_real, model, lpips_fn, 'real', device=device, batch_size=4)
    errors_fake, labels_fake = process_frames_from_folder(path_fake, model, lpips_fn, 'fake', device=device, batch_size=4)

    if errors_real.size == 0 and errors_fake.size == 0:
        raise RuntimeError("No errors computed. Check paths or model checkpoint.")

    X = np.vstack([errors_real, errors_fake])
    y = np.concatenate([labels_real, labels_fake])
    print(f"📊 Total samples: {len(X)} (Real: {len(errors_real)}, Fake: {len(errors_fake)})")

    # 🔸 Train classifier
    classifier, scaler = train_balanced_classifier(X, y, output_dir)

    joblib.dump(classifier, os.path.join(output_dir, 'deepfake_classifier_xgb.pkl'))
    joblib.dump(scaler, os.path.join(output_dir, 'scaler.pkl'))
    print(f"✅ Classifier and scaler saved in {output_dir}")


🔹 Loading e4e model...
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
🔹 Initializing LPIPS metric...
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\EliteLaptop/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:25<00:00, 9.42MB/s] 


Loading model from: c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\lpips\weights\v0.1\alex.pth
🔄 Processing 790 images for 'real' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real...


Processing real: 100%|██████████| 198/198 [06:04<00:00,  1.84s/it]


🔄 Processing 3975 images for 'fake' in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake...


Processing fake: 100%|██████████| 994/994 [24:51<00:00,  1.50s/it]  


📊 Total samples: 4765 (Real: 790, Fake: 3975)
🎯 Accuracy (train): 0.9605
🎯 Accuracy (test) : 0.8805
              precision    recall  f1-score   support

        Real       0.85      0.93      0.89       795
        Fake       0.92      0.83      0.87       795

    accuracy                           0.88      1590
   macro avg       0.88      0.88      0.88      1590
weighted avg       0.88      0.88      0.88      1590

✅ Classifier and scaler saved in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust_optimized


In [5]:
import os
import cv2
import torch
import numpy as np
import joblib
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
import lpips
from encoder4editing.models.psp import pSp
from argparse import Namespace
from skimage.metrics import structural_similarity as ssim
import torch.nn.functional as F

# ---------------------------------------------------------
# 🔧 Reprend les fonctions de ton script d’entraînement
# ---------------------------------------------------------
def tensor_to_numpy01(tensor):
    t = tensor.detach().cpu()
    if t.dim() == 3:
        arr = t.permute(1, 2, 0).numpy()
    elif t.dim() == 4:
        arr = t.permute(0, 2, 3, 1).numpy()
    else:
        raise ValueError("Tensor must be 3D or 4D")
    if arr.min() < -0.1 or arr.max() > 1.1:
        arr = np.clip(arr, -1.0, 1.0)
        arr = (arr + 1.0) / 2.0
    else:
        arr = np.clip(arr, 0.0, 1.0)
    return arr

def compute_reconstruction_errors(model, image_tensor, lpips_fn, device='cuda'):
    model.eval()
    with torch.no_grad():
        try:
            out = model(image_tensor, randomize_noise=False, resize=False)
        except TypeError:
            out = model(image_tensor)
        reconstructed = out[0] if isinstance(out, (tuple, list)) else out

    if reconstructed.shape != image_tensor.shape:
        reconstructed = torch.nn.functional.interpolate(
            reconstructed, size=image_tensor.shape[2:], mode='bilinear', align_corners=False
        )

    orig_np = tensor_to_numpy01(image_tensor)
    rec_np = tensor_to_numpy01(reconstructed)

    if orig_np.ndim == 3:
        orig_np = orig_np[np.newaxis, ...]
    if rec_np.ndim == 3:
        rec_np = rec_np[np.newaxis, ...]

    mse_loss = np.mean((orig_np - rec_np) ** 2, axis=(1, 2, 3))
    ssim_scores = [ssim(orig_np[i], rec_np[i], channel_axis=2, data_range=1.0) for i in range(orig_np.shape[0])]
    psnr_vals = [20 * np.log10(1.0 / np.sqrt(m + 1e-8)) for m in mse_loss]

    with torch.no_grad():
        lpips_vals = lpips_fn(image_tensor, reconstructed).detach().cpu().numpy().flatten()

    cosine_vals = [
        F.cosine_similarity(image_tensor[i].flatten(), reconstructed[i].flatten(), dim=0).item()
        for i in range(image_tensor.shape[0])
    ]

    errors = np.stack([
        mse_loss,
        1.0 - np.array(ssim_scores),
        np.array(psnr_vals),
        np.array(lpips_vals),
        1.0 - np.array(cosine_vals)
    ], axis=1)
    return errors


# ---------------------------------------------------------
# 🧩 Fonction pour extraire des frames d’une vidéo
# ---------------------------------------------------------
def extract_frames(video_path, output_size=(256, 256), frame_skip=10):
    cap = cv2.VideoCapture(video_path)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(output_size),
        transforms.ToTensor()
    ])
    frames = []
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_skip == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = transform(frame_rgb)
            frames.append(frame_tensor)
        count += 1
    cap.release()
    return frames


# ---------------------------------------------------------
# 🧠 Charger le modèle e4e + classifieur
# ---------------------------------------------------------
def load_e4e_model(ckpt_path, device='cuda'):
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device).eval()
    return model


# ---------------------------------------------------------
# 🚀 Prédiction sur une vidéo
# ---------------------------------------------------------
def predict_video(video_path, model, lpips_fn, clf, scaler, device='cuda', batch_size=4):
    print(f"🔍 Analyse de la vidéo : {video_path}")
    frames = extract_frames(video_path, frame_skip=10)
    if len(frames) == 0:
        print("❌ Aucune frame extraite.")
        return None

    all_errors = []
    for i in tqdm(range(0, len(frames), batch_size), desc="Processing frames"):
        batch = torch.stack(frames[i:i+batch_size]).to(device)
        errors = compute_reconstruction_errors(model, batch, lpips_fn, device)
        all_errors.append(errors)
    X = np.vstack(all_errors).astype(np.float32)

    X_scaled = scaler.transform(X)
    preds = clf.predict_proba(X_scaled)[:, 1]  # proba "fake"
    mean_score = np.mean(preds)

    print(f"📊 Moyenne probabilité 'fake' sur {len(preds)} frames : {mean_score:.4f}")
    verdict = "FAKE" if mean_score > 0.5 else "REAL"
    print(f"✅ Verdict global : {verdict}")
    return mean_score, verdict


# ---------------------------------------------------------
# 🏁 MAIN
# ---------------------------------------------------------
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
    model = load_e4e_model(ckpt_path, device)
    lpips_fn = lpips.LPIPS(net='alex').to(device)

    output_dir = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_Xgboust_optimized'
    clf = joblib.load(os.path.join(output_dir, 'deepfake_classifier_xgb.pkl'))
    scaler = joblib.load(os.path.join(output_dir, 'scaler.pkl'))

    # 🔸 Teste une vidéo
    video_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\fake\id0_id1_0002.mp4"
    predict_video(video_path, model, lpips_fn, clf, scaler, device)


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_24692\1999592791.py:106: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\lpips\weights\v0.1\alex.pth
🔍 Analyse de la vidéo : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\raw\fake\id0_id1_0002.mp4


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\lpips\lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path

📊 Moyenne probabilité 'fake' sur 35 frames : 0.7703
✅ Verdict global : FAKE


In [4]:
import os
import torch
from PIL import Image
from torchvision import transforms
from glob import glob
import numpy as np
from tqdm import tqdm

# ---------------------------------------------
# 🔹 Config
# ---------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 4  # taille du batch

# Chemins fournis
path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'
ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')

# Choisir quel dossier traiter
video_frames_folder = path_real  # ou path_fake
output_latents_file = "video_latents_real.npy"  # ou "video_latents_fake.npy"

# ---------------------------------------------
# 🔹 Load e4e / pSp model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device).eval()
    return model

model = load_model(ckpt_path, device=device)

# ---------------------------------------------
# 🔹 Transform for images
# ---------------------------------------------
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# ---------------------------------------------
# 🔹 Gather all frames (recursive)
# ---------------------------------------------
frame_paths = sorted(glob(os.path.join(video_frames_folder, '**', '*.*'), recursive=True))
frame_paths = [f for f in frame_paths if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Found {len(frame_paths)} frames in {video_frames_folder}.")

if len(frame_paths) == 0:
    raise RuntimeError(f"Aucune image trouvée dans {video_frames_folder} ! Vérifie les sous-dossiers et extensions.")

# ---------------------------------------------
# 🔹 Frame-by-frame GAN Inversion en batch
# ---------------------------------------------
all_latents = []
reconstructed_frames = []

for i in tqdm(range(0, len(frame_paths), batch_size), desc="Inverting frames"):
    batch_paths = frame_paths[i:i+batch_size]
    batch_imgs = []
    for path in batch_paths:
        try:
            img = Image.open(path).convert('RGB')
            img_tensor = transform(img)
            batch_imgs.append(img_tensor)
        except Exception as e:
            print(f"❌ Failed loading {path}: {e}")

    if not batch_imgs:
        continue

    batch_tensor = torch.stack(batch_imgs).to(device)

    with torch.no_grad():
        outputs = model(batch_tensor, return_latents=True)

        # Si sortie tuple (reconstruction, w)
        if isinstance(outputs, (tuple, list)):
            reconstructed, w = outputs
        else:
            w = outputs
            reconstructed = model.decoder(w, input_is_latent=True)[0]

        # Sauvegarder latents et reconstructions
        all_latents.append(w.cpu().numpy())
        reconstructed_frames.append(reconstructed.cpu().numpy())

    # libération VRAM entre les batchs
    del batch_tensor, batch_imgs, w, reconstructed
    torch.cuda.empty_cache()

# ---------------------------------------------
# 🔹 Vérification avant vstack
# ---------------------------------------------
if len(all_latents) == 0:
    raise RuntimeError("Aucun latent calculé. Vérifie tes images et ton modèle.")
else:
    all_latents = np.vstack(all_latents)
    print(f"Latents shape: {all_latents.shape}")

# ---------------------------------------------
# 🔹 Temporal variation analysis
# ---------------------------------------------
latent_variance = np.var(all_latents, axis=0)
mean_variance = np.mean(latent_variance)
print(f"Mean temporal variance of latent vectors: {mean_variance:.6f}")

# ---------------------------------------------
# 🔹 Save results
# ---------------------------------------------
np.save(output_latents_file, all_latents)
print(f"Latents saved to {output_latents_file}")


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
Found 790 frames in C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real.


Inverting frames: 100%|██████████| 198/198 [05:58<00:00,  1.81s/it]


Latents shape: (790, 18, 512)
Mean temporal variance of latent vectors: 0.094481
Latents saved to video_latents_real.npy


In [1]:
import os
import torch
from PIL import Image
from torchvision import transforms
from glob import glob
import numpy as np
from tqdm import tqdm
import torchvision.utils as vutils

# ---------------------------------------------
# 🔹 Config
# ---------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 4

# Dossiers des frames
path_real = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real'
path_fake = r'C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake'

# Dossiers de sauvegarde des reconstructions
recon_real = "./reconstruction_real"
recon_fake = "./reconstruction_fake"
os.makedirs(recon_real, exist_ok=True)
os.makedirs(recon_fake, exist_ok=True)

ckpt_path = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')

# ---------------------------------------------
# 🔹 Load e4e model
# ---------------------------------------------
def load_model(ckpt_path, device='cuda'):
    from encoder4editing.models.psp import pSp
    from argparse import Namespace
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=True)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device).eval()
    return model

model = load_model(ckpt_path, device=device)

# ---------------------------------------------
# 🔹 Transform for images
# ---------------------------------------------
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# ---------------------------------------------
# 🔹 Function to invert a video (real or fake)
# ---------------------------------------------
def process_video(video_frames_folder, output_latents_file, recon_output_folder):

    print(f"\n==============================")
    print(f" Processing: {video_frames_folder} ")
    print(f"==============================")

    # Gather frames
    frame_paths = sorted(glob(os.path.join(video_frames_folder, '**', '*.*'), recursive=True))
    frame_paths = [f for f in frame_paths if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"Found {len(frame_paths)} frames.")

    if len(frame_paths) == 0:
        raise RuntimeError("⚠ Aucun frame trouvé !")

    all_latents = []

    # Batch processing
    for i in tqdm(range(0, len(frame_paths), batch_size), desc="Inverting frames"):
        batch_paths = frame_paths[i:i + batch_size]
        batch_imgs = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert('RGB')
                img_tensor = transform(img)
                batch_imgs.append(img_tensor)
            except:
                print(f"❌ Failed loading: {path}")

        if not batch_imgs:
            continue

        batch_tensor = torch.stack(batch_imgs).to(device)

        with torch.no_grad():
            outputs = model(batch_tensor, return_latents=True)

            # True GAN inversion: Encoder → W+ → Generator
            if isinstance(outputs, (tuple, list)):
                reconstructed, w = outputs
            else:
                w = outputs
                reconstructed = model.decoder(w, input_is_latent=True)[0]

        # 🔹 Save reconstructed images (GAN reconstruction)
        for n in range(reconstructed.shape[0]):
            filename = os.path.basename(batch_paths[n])
            save_path = os.path.join(recon_output_folder, filename)
            vutils.save_image(reconstructed[n], save_path, normalize=True)

        all_latents.append(w.cpu().numpy())

        del batch_tensor, batch_imgs, w, reconstructed
        torch.cuda.empty_cache()

    # Concatenate
    all_latents = np.vstack(all_latents)
    print(f"Latents shape: {all_latents.shape}")

    # Temporal variance
    latent_variance = np.var(all_latents, axis=0)
    mean_variance = float(np.mean(latent_variance))
    print(f"Mean temporal variance: {mean_variance:.6f}")

    # Save
    np.save(output_latents_file, all_latents)
    print(f"Latents saved → {output_latents_file}")

    return mean_variance


# ---------------------------------------------
# 🔹 Run for REAL and FAKE
# ---------------------------------------------
real_variance = process_video(path_real, "video_latents_real.npy", recon_real)
fake_variance = process_video(path_fake, "video_latents_fake.npy", recon_fake)

print("\n==============================")
print("     🔍 FINAL COMPARISON      ")
print("==============================")
print(f"Real video variance  : {real_variance:.6f}")
print(f"Fake video variance  : {fake_variance:.6f}")

if fake_variance < real_variance * 0.7:
    print("\n🟢 Fake vidéo détectée : variance faible → très stable → GAN signature")
else:
    print("\n🔵 Les deux vidéos semblent naturelles → pas de signature GAN claire.")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path


 Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real 
Found 790 frames.


Inverting frames: 100%|██████████| 198/198 [04:54<00:00,  1.49s/it]


Latents shape: (790, 18, 512)
Mean temporal variance: 0.094481
Latents saved → video_latents_real.npy

 Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake 
Found 3975 frames.


Inverting frames: 100%|██████████| 994/994 [22:50<00:00,  1.38s/it]


Latents shape: (3975, 18, 512)
Mean temporal variance: 0.090602
Latents saved → video_latents_fake.npy

     🔍 FINAL COMPARISON      
Real video variance  : 0.094481
Fake video variance  : 0.090602

🔵 Les deux vidéos semblent naturelles → pas de signature GAN claire.


In [1]:
import sys
import os
import torch
from torch import nn, optim
from PIL import Image
from torchvision import transforms, utils
from glob import glob
import numpy as np
from tqdm import tqdm

# ============================================================
# 🔹 Add StyleGAN2-ADA-PyTorch folder to Python path
# ============================================================
repo_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ada-pytorch"
sys.path.insert(0, repo_path)

import dnnlib
import legacy

# ============================================================
# 🔹 CONFIGURATION
# ============================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
steps = 100
lr = 0.05
batch_size = 4  # 🔹 Utilisation du batch

# Paths
path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

recon_real = "./reconstruction_real"
recon_fake = "./reconstruction_fake"
os.makedirs(recon_real, exist_ok=True)
os.makedirs(recon_fake, exist_ok=True)

ckpt_path = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ffhq-config-f.pkl"

# ============================================================
# 🔹 LOAD STYLEGAN2 GENERATOR (G_ema)
# ============================================================
print("🔹 Loading StyleGAN2 generator...")
with open(ckpt_path, "rb") as f:
    G = legacy.load_network_pkl(f)["G_ema"].to(device)
G.eval()
print("✔ Generator loaded")

# ============================================================
# 🔹 TRANSFORMER
# ============================================================
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
])

# ============================================================
# 🔹 INVERSION : optimize W latent using gradient descent (batch)
# ============================================================
def invert_images_batch(img_tensors):
    """
    img_tensors: Tensor of shape [B, 3, 1024, 1024]
    Returns:
        ws: [B, num_ws, 512]
        recon_imgs: [B, 3, 1024, 1024]
    """
    B = img_tensors.size(0)
    img_tensors = img_tensors.to(device)

    with torch.no_grad():
        w_opt = G.mapping.w_avg.repeat(B, G.num_ws, 1)

    w_opt = w_opt.clone().detach().requires_grad_(True)
    optimizer = optim.Adam([w_opt], lr=lr)
    loss_fn = nn.MSELoss()

    for step in range(steps):
        optimizer.zero_grad()
        synth = G.synthesis(w_opt, noise_mode="const")
        synth = (synth + 1) / 2
        loss = loss_fn(synth, img_tensors)
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}/{steps} - Loss: {loss.item():.4f}")

    with torch.no_grad():
        final_imgs = G.synthesis(w_opt, noise_mode="const")
        final_imgs = (final_imgs + 1) / 2

    return w_opt.detach().cpu().numpy(), final_imgs.detach()

# ============================================================
# 🔹 PROCESS VIDEO FRAMES IN BATCH
# ============================================================
def process_video(folder, save_npy, save_recon):
    print(f"\n==============================")
    print(f" Processing: {folder} ")
    print(f"==============================")

    # Recursive glob for nested subfolders
    frame_paths = sorted(glob(os.path.join(folder, "**", "*.*"), recursive=True))
    frame_paths = [f for f in frame_paths if f.lower().endswith(("jpg", "jpeg", "png"))]

    if len(frame_paths) == 0:
        raise RuntimeError("❌ Aucun frame trouvé !")

    print(f"✔ Found {len(frame_paths)} frames.")

    all_latents = []

    # Traitement par batch
    for i in tqdm(range(0, len(frame_paths), batch_size), desc="Inverting"):
        batch_paths = frame_paths[i:i+batch_size]
        imgs = [transform(Image.open(p).convert("RGB")) for p in batch_paths]
        img_tensors = torch.stack(imgs)  # [B, 3, 1024, 1024]

        ws_batch, recon_imgs_batch = invert_images_batch(img_tensors)
        all_latents.extend(ws_batch)

        # Save reconstructed images
        for j, p in enumerate(batch_paths):
            filename = os.path.basename(p)
            save_path = os.path.join(save_recon, filename)
            utils.save_image(recon_imgs_batch[j], save_path, normalize=True)

    all_latents = np.array(all_latents)
    np.save(save_npy, all_latents)
    print("✔ Latents saved:", save_npy)

    variance = np.mean(np.var(all_latents, axis=0))
    print(f"Temporal variance: {variance:.6f}")
    return variance

# ============================================================
# 🔹 RUN ON REAL & FAKE VIDEO FRAMES
# ============================================================
var_real = process_video(path_real, "video_latents_real.npy", recon_real)
var_fake = process_video(path_fake, "video_latents_fake.npy", recon_fake)

print("\n==============================")
print("        🔍 FINAL REPORT")
print("==============================")
print(f"Variance REAL : {var_real:.6f}")
print(f"Variance FAKE : {var_fake:.6f}")

if var_fake < var_real * 0.7:
    print("\n🟢 Fake détectée : variance faible → GAN signature")
else:
    print("\n🔵 Vidéo naturelle : pas de signature GAN claire")


🔹 Loading StyleGAN2 generator...
✔ Generator loaded

 Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real 
✔ Found 790 frames.


Inverting:   0%|          | 0/198 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... 

c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... 

...

Step 0/100 - Loss: 0.1468
Step 50/100 - Loss: 0.0187


Inverting:   3%|▎         | 5/198 [1:32:51<59:52:12, 1116.75s/it]

Step 0/100 - Loss: 0.1547


Inverting:   3%|▎         | 5/198 [1:41:04<65:01:16, 1212.83s/it]


KeyboardInterrupt: 

In [8]:
import sys
import os
import torch
from PIL import Image
import numpy as np
from glob import glob
from tqdm import tqdm
from torchvision import transforms, utils

# ---------------------------
# Paths (ADJUST if needed)
# ---------------------------
stylegan_repo = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ada-pytorch"
encoder_repo = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing"  # folder containing encoder4editing package
e4e_ckpt = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt"
stylegan_ckpt = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ffhq-config-f.pkl"

# Frame folders
path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

# Output folders
recon_real = "./recon_real"
recon_fake = "./recon_fake"
os.makedirs(recon_real, exist_ok=True)
os.makedirs(recon_fake, exist_ok=True)

# ---------------------------
# Add repos to PYTHONPATH
# ---------------------------
if os.path.isdir(stylegan_repo):
    sys.path.insert(0, stylegan_repo)
else:
    raise FileNotFoundError(f"StyleGAN repo not found at: {stylegan_repo}")

if os.path.isdir(encoder_repo):
    sys.path.insert(0, encoder_repo)
else:
    raise FileNotFoundError(f"Encoder4Editing repo not found at: {encoder_repo}")

# ---------------------------
# Imports that depend on repos
# ---------------------------
import dnnlib
import legacy

# We'll import pSp (e4e model)
try:
    from encoder4editing.models.psp import pSp
except Exception as e:
    # give a clear error if the import fails
    raise ImportError("Failed to import pSp from encoder4editing. Make sure the encoder4editing repo is the correct one and in PYTHONPATH.") from e

from argparse import Namespace

# ---------------------------
# Device & transforms
# ---------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Torch version: {torch.__version__} | CUDA available: {torch.cuda.is_available()} | Device: {device}")

img_res = 256  # e4e expects smaller input for encoder preprocessing (we use 256)
transform = transforms.Compose([
    transforms.Resize((img_res, img_res)),
    transforms.ToTensor()
])

# ---------------------------
# Load StyleGAN2 generator
# ---------------------------
print("Loading StyleGAN2 generator from:", stylegan_ckpt)
if not os.path.isfile(stylegan_ckpt):
    raise FileNotFoundError(f"StyleGAN2 checkpoint not found at: {stylegan_ckpt}")

with open(stylegan_ckpt, "rb") as f:
    G = legacy.load_network_pkl(f)["G_ema"].to(device)
G.eval()
print("✔ StyleGAN2 loaded (G_ema)")

# ---------------------------
# Load e4e encoder (pSp)
# ---------------------------
def load_e4e(ckpt_path, device='cuda'):
    if not os.path.isfile(ckpt_path):
        raise FileNotFoundError(f"e4e checkpoint not found at: {ckpt_path}")
    ckpt = torch.load(ckpt_path, map_location=device)
    if 'opts' not in ckpt:
        # Some saved checkpoints may have different key names; attempt to handle common cases
        if 'state_dict' in ckpt and 'opts' not in ckpt:
            raise RuntimeError("Loaded checkpoint does not contain 'opts' required to build pSp. Make sure you use the e4e pretrained checkpoint from encoder4editing repository.")
        else:
            raise RuntimeError("Checkpoint format unexpected: missing 'opts'.")
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts)
    model.to(device)
    # load weights
    try:
        state_dict = ckpt.get('state_dict', ckpt.get('model_state_dict', None))
        if state_dict is None:
            # some checkpoints store weights at top level
            model.load_state_dict(ckpt, strict=False)
        else:
            model.load_state_dict(state_dict, strict=False)
    except Exception as e:
        # load with partial strictness if shapes/names mismatch
        model.load_state_dict(ckpt.get('state_dict', {}), strict=False)
    model.eval()
    return model

print("Loading e4e encoder from:", e4e_ckpt)
encoder = load_e4e(e4e_ckpt, device=device)
print("✔ e4e encoder loaded")

# ---------------------------
# Inversion helper using e4e
# ---------------------------
@torch.no_grad()
def invert_e4e(img_tensor):
    """
    img_tensor: [1,3,H,W] float tensor in [0,1], already resized to img_res
    returns: w (numpy) and reconstruction tensor [1,3,Hg,Wg] (generator output)
    """
    img_tensor = img_tensor.to(device)

    # Some pSp/e4e models expect input normalized to [-1,1]. Confirm and convert if required.
    # Many encoder4editing implementations internally normalize; if not, uncomment the next line:
    # x_in = img_tensor * 2 - 1
    x_in = img_tensor  # keep as [0,1] — the pSp in encoder4editing repo usually handles preprocessing

    outputs = encoder(x_in, return_latents=True)
    # encoder may return (recon_images, w) or dict-like; handle common patterns:
    if isinstance(outputs, (tuple, list)):
        # common: (reconstructed_img, w)
        if len(outputs) == 2:
            recon_from_enc, w = outputs
        else:
            # fallback: try last element as w
            w = outputs[-1]
            recon_from_enc = None
    elif isinstance(outputs, dict):
        w = outputs.get('w', outputs.get('latent', None))
        recon_from_enc = outputs.get('image', None)
    else:
        # unknown API: assume encoder returns w directly
        w = outputs
        recon_from_enc = None

    # ensure w shape: [1, num_ws, w_dim] or [1, w_dim]
    if isinstance(w, torch.Tensor) and w.ndim == 2:
        # shape [1, w_dim] -> expand to W+ if needed
        w = w.unsqueeze(1).repeat(1, G.num_ws, 1)

    # generate with StyleGAN
    with torch.no_grad():
        synth = G.synthesis(w, noise_mode='const')
        synth = (synth + 1) / 2  # to [0,1]

    return w.cpu().numpy(), synth.cpu()

# ---------------------------
# Process video frames
# ---------------------------
def process_video(folder, save_npy, save_recon):
    print("\n==============================")
    print(" Processing:", folder)
    print("==============================")

    frame_paths = sorted(glob(os.path.join(folder, "**", "*.*"), recursive=True))
    frame_paths = [p for p in frame_paths if p.lower().endswith((".jpg", ".jpeg", ".png"))]
    print("Found", len(frame_paths), "frames")

    if len(frame_paths) == 0:
        raise RuntimeError("No frames found in folder: " + folder)

    os.makedirs(save_recon, exist_ok=True)
    all_latents = []

    for p in tqdm(frame_paths, desc="Inverting"):
        try:
            img = Image.open(p).convert("RGB")
        except Exception as e:
            print("Failed to open:", p, "|", e)
            continue

        img_t = transform(img).unsqueeze(0)  # [1,3,H,W]
        w, recon = invert_e4e(img_t)         # w: numpy, recon: tensor [1,3,Hg,Wg]

        all_latents.append(w[0])              # store single example

        # Save reconstruction as image
        save_path = os.path.join(save_recon, os.path.basename(p))
        utils.save_image(recon[0], save_path)

    all_latents = np.array(all_latents)  # shape (N, num_ws, w_dim) or (N, w_dim)
    np.save(save_npy, all_latents)
    print("Saved latents to:", save_npy)

    # temporal variance
    latent_variance = np.var(all_latents, axis=0)
    mean_variance = float(np.mean(latent_variance))
    print("Mean temporal variance:", mean_variance)
    return mean_variance

# ---------------------------
# Run for real and fake
# ---------------------------
var_real = process_video(path_real, "video_latents_real.npy", recon_real)
var_fake = process_video(path_fake, "video_latents_fake.npy", recon_fake)

print("\n===== FINAL REPORT =====")
print("Real variance :", var_real)
print("Fake variance :", var_fake)
if var_fake < var_real * 0.7:
    print("=> GAN detected (fake).")
else:
    print("=> Looks natural.")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Torch version: 2.5.1 | CUDA available: True | Device: cuda
Loading StyleGAN2 generator from: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ffhq-config-f.pkl
✔ StyleGAN2 loaded (G_ema)
Loading e4e encoder from: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_6540\4033722950.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location=device)


Loading e4e over the pSp framework from checkpoint: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

✔ e4e encoder loaded

 Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real
Found 790 frames


Inverting: 100%|██████████| 790/790 [06:05<00:00,  2.16it/s]


Saved latents to: video_latents_real.npy
Mean temporal variance: 0.09448085725307465

 Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake
Found 3975 frames


Inverting: 100%|██████████| 3975/3975 [33:44<00:00,  1.96it/s]   


Saved latents to: video_latents_fake.npy
Mean temporal variance: 0.09060238301753998

===== FINAL REPORT =====
Real variance : 0.09448085725307465
Fake variance : 0.09060238301753998
=> Looks natural.


In [1]:
import sys
import os
import torch
from PIL import Image
import numpy as np
from glob import glob
from tqdm import tqdm
from torchvision import transforms, utils

# ---------------------------
# Paths
# ---------------------------
stylegan_repo = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ada-pytorch"
encoder_repo = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing"
e4e_ckpt = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt"
stylegan_ckpt = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\stylegan2-ffhq-config-f.pkl"

path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

recon_real = "./recon_real"
recon_fake = "./recon_fake"
os.makedirs(recon_real, exist_ok=True)
os.makedirs(recon_fake, exist_ok=True)

# ---------------------------
# Add repos to PYTHONPATH  (FIXED)
# ---------------------------
# IMPORTANT : StyleGAN MUST come FIRST to avoid import conflict
sys.path.insert(0, stylegan_repo)

# encoder4editing goes AFTER to prevent overriding 'training'
sys.path.append(encoder_repo)

import dnnlib
import legacy
from encoder4editing.models.psp import pSp
from argparse import Namespace

# ---------------------------
# Device & transforms
# ---------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
img_res = 256
transform = transforms.Compose([
    transforms.Resize((img_res, img_res)),
    transforms.ToTensor()
])

# ---------------------------
# Load StyleGAN2
# ---------------------------
with open(stylegan_ckpt, "rb") as f:
    G = legacy.load_network_pkl(f)["G_ema"].to(device)
G.eval()

# ---------------------------
# Load e4e encoder
# ---------------------------
def load_e4e(ckpt_path, device='cuda'):
    ckpt = torch.load(ckpt_path, map_location=device)
    opts = Namespace(**ckpt['opts'])
    opts.checkpoint_path = ckpt_path
    model = pSp(opts).to(device)
    state_dict = ckpt.get('state_dict', ckpt.get('model_state_dict', None))
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model

encoder = load_e4e(e4e_ckpt, device=device)

# ---------------------------
# Helper: recadrage central
# ---------------------------
def crop_center(img, size=256):
    w, h = img.size
    left = max((w - size) // 2, 0)
    top = max((h - size) // 2, 0)
    right = min(left + size, w)
    bottom = min(top + size, h)
    return img.crop((left, top, right, bottom))

# ---------------------------
# Inversion e4e
# ---------------------------
@torch.no_grad()
def invert_e4e(img_tensor):
    img_tensor = img_tensor.to(device)
    outputs = encoder(img_tensor, return_latents=True)

    if isinstance(outputs, (tuple, list)) and len(outputs) == 2:
        recon_from_enc, w = outputs
    elif isinstance(outputs, dict):
        w = outputs.get('w', outputs.get('latent'))
        recon_from_enc = outputs.get('image', None)
    else:
        w = outputs
        recon_from_enc = None

    if w.ndim == 2:
        w = w.unsqueeze(1).repeat(1, G.num_ws, 1)

    synth = G.synthesis(w, noise_mode='const')
    synth = (synth + 1) / 2

    return w.cpu().float().numpy(), synth.cpu()

# ---------------------------
# Traitement des frames par batch
# ---------------------------
def process_video(folder, save_npy, save_recon, batch_size=4):
    print("\nProcessing:", folder)
    frame_paths = sorted(glob(os.path.join(folder, "**", "*.*"), recursive=True))
    frame_paths = [p for p in frame_paths if p.lower().endswith((".jpg", ".jpeg", ".png"))]
    os.makedirs(save_recon, exist_ok=True)

    all_latents = []

    for i in tqdm(range(0, len(frame_paths), batch_size), desc="Inverting"):
        batch_paths = frame_paths[i:i+batch_size]
        batch_imgs = []

        for p in batch_paths:
            try:
                img = Image.open(p).convert("RGB")
                img = crop_center(img, size=img_res)
                batch_imgs.append(transform(img))
            except:
                continue

        if len(batch_imgs) == 0:
            continue

        batch_tensor = torch.stack(batch_imgs, dim=0).to(device)
        with torch.no_grad():
            outputs = encoder(batch_tensor, return_latents=True)
            if isinstance(outputs, (tuple, list)) and len(outputs) == 2:
                recon_batch, w_batch = outputs
            elif isinstance(outputs, dict):
                w_batch = outputs.get('w', outputs.get('latent'))
                recon_batch = outputs.get('image', None)
            else:
                w_batch = outputs
                recon_batch = None

            if w_batch.ndim == 2:
                w_batch = w_batch.unsqueeze(1).repeat(1, G.num_ws, 1)

            synth_batch = G.synthesis(w_batch, noise_mode='const')
            synth_batch = (synth_batch + 1) / 2

        for idx, p in enumerate(batch_paths):
            latent = w_batch[idx].cpu().numpy()
            if np.isnan(latent).any() or np.isinf(latent).any():
                continue
            all_latents.append(latent)
            utils.save_image(synth_batch[idx].cpu(), os.path.join(save_recon, os.path.basename(p)))

    if len(all_latents) == 0:
        print("No valid latents found in folder:", folder)
        return float('nan')

    all_latents = np.array(all_latents)
    np.save(save_npy, all_latents)

    latent_variance = np.var(all_latents, axis=0)
    mean_variance = float(np.mean(latent_variance))
    print("Mean temporal variance:", mean_variance)
    return mean_variance

# ---------------------------
# Exécution
# ---------------------------
var_real = process_video(path_real, "video_latents_real.npy", recon_real, batch_size=4)
var_fake = process_video(path_fake, "video_latents_fake.npy", recon_fake, batch_size=4)

print("\n===== FINAL REPORT =====")
print("Real variance :", var_real)
print("Fake variance :", var_fake)
if not np.isnan(var_real) and not np.isnan(var_fake):
    if var_fake < var_real * 0.7:
        print("=> GAN detected (fake).")
    else:
        print("=> Looks natural.")
else:
    print("=> Unable to determine: NaN values detected.")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_16308\2697679075.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.s

Loading e4e over the pSp framework from checkpoint: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path


Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real


Inverting:   0%|          | 0/198 [00:00<?, ?it/s]

...

Done.


Inverting: 100%|██████████| 198/198 [1:39:21<00:00, 30.11s/it] 


Mean temporal variance: 0.14745275676250458

Processing: C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake


Inverting: 100%|██████████| 994/994 [7:42:49<00:00, 27.94s/it]     


Mean temporal variance: 0.14742860198020935

===== FINAL REPORT =====
Real variance : 0.14745275676250458
Fake variance : 0.14742860198020935
=> Looks natural.


In [3]:
!pip install facenet-pytorch


In [5]:
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install lpips


  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires numpy<2.0.0,>=1.24.0, but you have numpy 2.0.2 which is incompatible.


In [2]:
!pip uninstall -y numpy


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


You can safely remove it manually.
You can safely remove it manually.


In [3]:
!pip install numpy==1.26.4


  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [3]:
# ==========================================================
# NOTEBOOK UNIQUE : Deepfake detection pipeline
# ==========================================================
# Requirements:
# pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import joblib
import json

# sklearn & others
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from scipy.spatial.distance import cdist

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# Tes dossiers contenant des frames DÉJÀ extraites
path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

# checkpoint e4e si disponible
ckpt_e4e = os.path.join('encoder4editing','pretrained_models','e4e_ffhq_encode.pt')

# dossier de sortie
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_pipeline"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True     # True = essayer e4e, sinon fallback ResNet50

# ----------------------------------------------------------
# CHARGEMENT DU MODELE : e4e OU RESNET50
# ----------------------------------------------------------
e4e_model = None
resnet_model = None

transform_resnet = T.Compose([
    T.ToPILImage(),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# tentative e4e
def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()

        def encode(img_bgr):
            img_rgb = img_bgr[:,:,::-1]
            tf = T.Compose([T.ToPILImage(), T.Resize((256,256)), T.ToTensor()])
            t = tf(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
                return out.cpu().numpy().reshape(-1)
        print("e4e loaded successfully.")
        return encode
    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None

def load_resnet_fallback():
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Identity()
    model.to(device).eval()
    def encode(img_bgr):
        img_rgb = img_bgr[:,:,::-1]
        t = transform_resnet(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            return model(t).cpu().numpy().reshape(-1)
    return encode

if use_gan_inversion and os.path.exists(ckpt_e4e):
    e4e_model = try_load_e4e(ckpt_e4e)

encoder_fn = e4e_model if e4e_model is not None else load_resnet_fallback()
print("Using encoder:", "e4e" if e4e_model else "ResNet50")

# ----------------------------------------------------------
# 1) LECTURE DES IMAGES (supporte les sous-dossiers)
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    all_files = []
    for root, dirs, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(exts):
                all_files.append(os.path.join(root, f))
    return all_files

imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)

print("Found REAL:", len(imgs_real))
print("Found FAKE:", len(imgs_fake))

# ----------------------------------------------------------
# 2) EXTRACTION DES FEATURES / LATENTS
# ----------------------------------------------------------
def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError("Unable to read image " + p)
    return img

X_list = []
y_list = []

for p in tqdm(imgs_real, desc="Encoding REAL"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(0)
    except Exception as e:
        print("Error:", p, e)

for p in tqdm(imgs_fake, desc="Encoding FAKE"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(1)
    except Exception as e:
        print("Error:", p, e)

X = np.array(X_list)
y = np.array(y_list)

print("Latents shape:", X.shape)

np.save(os.path.join(output_dir, "latent_vectors.npy"), X)
np.save(os.path.join(output_dir, "labels.npy"), y)

# ----------------------------------------------------------
# 3) ANALYSE DE L’ESPACE LATENT
# ----------------------------------------------------------
results_summary = {}

# PCA
if X.shape[0] >= 2:
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    plt.scatter(X_pca[:,0], X_pca[:,1], c=y)
    plt.title("PCA 2D")
    plt.savefig(os.path.join(output_dir, "pca.png"))
    plt.close()

# t-SNE
if X.shape[0] >= 5:
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X)
    plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)
    plt.title("t-SNE")
    plt.savefig(os.path.join(output_dir, "tsne.png"))
    plt.close()

# UMAP
if X.shape[0] >= 5:
    reducer = umap.UMAP(n_components=2, random_state=42)
    X_umap = reducer.fit_transform(X)
    plt.scatter(X_umap[:,0], X_umap[:,1], c=y)
    plt.title("UMAP")
    plt.savefig(os.path.join(output_dir, "umap.png"))
    plt.close()

# distances intra/inter
if len(np.unique(y)) == 2:
    X_real = X[y==0]
    X_fake = X[y==1]

    dist_real = np.mean(cdist(X_real, X_real)) if len(X_real)>=2 else None
    dist_fake = np.mean(cdist(X_fake, X_fake)) if len(X_fake)>=2 else None
    dist_inter = np.mean(cdist(X_real, X_fake)) if len(X_real)>=1 and len(X_fake)>=1 else None

    results_summary.update({
        'intra_real': dist_real,
        'intra_fake': dist_fake,
        'inter': dist_inter
    })

with open(os.path.join(output_dir, "latent_analysis.json"), "w") as f:
    json.dump(results_summary, f, indent=2)

# ----------------------------------------------------------
# 4) MACHINE LEARNING
# ----------------------------------------------------------
if X.shape[0] < 10:
    print("Pas assez d’échantillons pour ML !")
else:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, stratify=y, random_state=42
    )

    models_results = {}

    # SVM
    svm = SVC(kernel='rbf')
    svm.fit(X_train, y_train)
    pred = svm.predict(X_test)
    models_results["svm"] = float(accuracy_score(y_test, pred))
    joblib.dump(svm, os.path.join(output_dir, "model_svm.joblib"))

    # Random Forest
    rf = RandomForestClassifier(n_estimators=200)
    rf.fit(X_train, y_train)
    pred = rf.predict(X_test)
    models_results["rf"] = float(accuracy_score(y_test, pred))
    joblib.dump(rf, os.path.join(output_dir, "model_rf.joblib"))

    # MLP
    mlp = MLPClassifier(hidden_layer_sizes=(512,256), max_iter=500)
    mlp.fit(X_train, y_train)
    pred = mlp.predict(X_test)
    models_results["mlp"] = float(accuracy_score(y_test, pred))
    joblib.dump(mlp, os.path.join(output_dir, "model_mlp.joblib"))

    # XGBoost
    xgb = XGBClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=6,
        eval_metric="logloss"
    )
    xgb.fit(X_train, y_train)
    pred = xgb.predict(X_test)
    models_results["xgb"] = float(accuracy_score(y_test, pred))
    joblib.dump(xgb, os.path.join(output_dir, "model_xgb.joblib"))

    joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))

    with open(os.path.join(output_dir, "results_models.json"), "w") as f:
        json.dump(models_results, f, indent=2)

    print("\nAccuracy des modèles :")
    for m,a in models_results.items():
        print(f" - {m}: {a:.4f}")

# ----------------------------------------------------------
# FIN
# ----------------------------------------------------------
print("\nPipeline terminé ! Résultats dans :", output_dir)


Device: cuda


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_9820\905531602.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.ser

Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

e4e loaded successfully.
Using encoder: e4e
Found REAL: 790
Found FAKE: 3975


Encoding FAKE: 100%|██████████| 3975/3975 [06:59<00:00,  9.48it/s]


Latents shape: (4765, 9216)


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(



Accuracy des modèles :
 - svm: 0.8395
 - rf: 0.8353
 - mlp: 0.8468
 - xgb: 0.8321

Pipeline terminé ! Résultats dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_pipeline


In [8]:
!pip install albumentations


  Using cached albumentations-2.0.8-py3-none-any.whl.metadata (43 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached albucore-0.0.24-py3-none-any.whl.metadata (5.3 kB)
  Using cached eval_type_backport-0.3.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached stringzilla-4.4.0.tar.gz (549 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached simsimd-6.5.3-cp39-cp39-win_amd64.whl.metadata (71 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp39-cp39-w

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\EliteLaptop\\miniconda3\\envs\\torch_gpu\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
# ==========================================================
# Deepfake detection pipeline - VERSION OPTIMISÉE
# ==========================================================
# Requirements:
# pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python imbalanced-learn albumentations

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import joblib
import json
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.spatial.distance import cdist
from imblearn.over_sampling import SMOTE
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

ckpt_e4e = os.path.join('encoder4editing','pretrained_models','e4e_ffhq_encode.pt')
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_pipeline_optimise"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True

# ----------------------------------------------------------
# CHARGEMENT MODELE
# ----------------------------------------------------------
e4e_model = None

transform_resnet = T.Compose([
    T.ToPILImage(),
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()
        def encode(img_bgr):
            img_rgb = img_bgr[:,:,::-1]
            tf = T.Compose([T.ToPILImage(), T.Resize((256,256)), T.ToTensor()])
            t = tf(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
                return out.cpu().numpy().reshape(-1)
        print("e4e loaded successfully.")
        return encode
    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None

def load_resnet_fallback():
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Identity()
    model.to(device).eval()
    def encode(img_bgr):
        img_rgb = img_bgr[:,:,::-1]
        t = transform_resnet(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            return model(t).cpu().numpy().reshape(-1)
    return encode

if use_gan_inversion and os.path.exists(ckpt_e4e):
    e4e_model = try_load_e4e(ckpt_e4e)

encoder_fn = e4e_model if e4e_model is not None else load_resnet_fallback()
print("Using encoder:", "e4e" if e4e_model else "ResNet50")

# ----------------------------------------------------------
# LISTE DES IMAGES
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    all_files = []
    for root, dirs, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(exts):
                all_files.append(os.path.join(root, f))
    return all_files

imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)
print("Found REAL:", len(imgs_real))
print("Found FAKE:", len(imgs_fake))

# ----------------------------------------------------------
# AUGMENTATION DATA REAL
# ----------------------------------------------------------
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
])

def augment_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    augmented = augment(image=img)['image']
    return cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)

# ----------------------------------------------------------
# EXTRACTION DES LATENTS
# ----------------------------------------------------------
def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError("Unable to read image " + p)
    return img

X_list = []
y_list = []

for p in tqdm(imgs_real, desc="Encoding REAL"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(0)
        # Augmentation 1x pour REAL
        img_aug = augment_image(img)
        X_list.append(encoder_fn(img_aug))
        y_list.append(0)
    except Exception as e:
        print("Error:", p, e)

for p in tqdm(imgs_fake, desc="Encoding FAKE"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(1)
    except Exception as e:
        print("Error:", p, e)

X = np.array(X_list)
y = np.array(y_list)
print("Latents shape:", X.shape)
np.save(os.path.join(output_dir, "latent_vectors.npy"), X)
np.save(os.path.join(output_dir, "labels.npy"), y)

# ----------------------------------------------------------
# REDUCTION DE DIMENSION POUR ML
# ----------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=512, random_state=42)
X_reduced = pca.fit_transform(X_scaled)
joblib.dump(pca, os.path.join(output_dir, "pca.joblib"))

# ----------------------------------------------------------
# ÉQUILIBRAGE DATASET
# ----------------------------------------------------------
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_reduced, y)
print("Dataset après SMOTE :", np.bincount(y_res))

# ----------------------------------------------------------
# TRAIN / TEST SPLIT
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, stratify=y_res, random_state=42
)

# ----------------------------------------------------------
# MODELES AVEC HYPERPARAM OPTIMISÉS
# ----------------------------------------------------------
models_results = {}

# SVM
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)
models_results["svm"] = float(accuracy_score(y_test, pred_svm))

# Random Forest
rf = RandomForestClassifier(n_estimators=500, max_depth=None, random_state=42)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
models_results["rf"] = float(accuracy_score(y_test, pred_rf))

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700, activation='relu', random_state=42)
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)
models_results["mlp"] = float(accuracy_score(y_test, pred_mlp))

# XGBoost
ratio = np.sum(y_res==0)/np.sum(y_res==1)
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    eval_metric="logloss",
    scale_pos_weight=ratio,
    random_state=42
)
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
models_results["xgb"] = float(accuracy_score(y_test, pred_xgb))

# ----------------------------------------------------------
# VotingClassifier (ensemble)
# ----------------------------------------------------------
voting = VotingClassifier(estimators=[
    ('rf', rf),
    ('mlp', mlp),
    ('xgb', xgb)
], voting='soft')

voting.fit(X_train, y_train)
pred_voting = voting.predict(X_test)
models_results["voting"] = float(accuracy_score(y_test, pred_voting))

# ----------------------------------------------------------
# SAVE MODELS ET RESULTATS
# ----------------------------------------------------------
joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))
joblib.dump(svm, os.path.join(output_dir, "model_svm.joblib"))
joblib.dump(rf, os.path.join(output_dir, "model_rf.joblib"))
joblib.dump(mlp, os.path.join(output_dir, "model_mlp.joblib"))
joblib.dump(xgb, os.path.join(output_dir, "model_xgb.joblib"))
joblib.dump(voting, os.path.join(output_dir, "model_voting.joblib"))

with open(os.path.join(output_dir, "results_models.json"), "w") as f:
    json.dump(models_results, f, indent=2)

print("\nAccuracy des modèles :")
for m,a in models_results.items():
    print(f" - {m}: {a:.4f}")

print("\nPipeline terminé ! Résultats dans :", output_dir)


Device: cuda


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_15792\1003403170.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

e4e loaded successfully.
Using encoder: e4e
Found REAL: 790
Found FAKE: 3975


Encoding FAKE: 100%|██████████| 3975/3975 [07:06<00:00,  9.31it/s]


Latents shape: (5555, 9216)


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Dataset après SMOTE : [3975 3975]

Accuracy des modèles :
 - svm: 0.8119
 - rf: 0.8535
 - mlp: 0.9308
 - xgb: 0.8736
 - voting: 0.9283

Pipeline terminé ! Résultats dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_gan_pipeline_optimise


In [10]:
# ==========================================================
# Deepfake detection pipeline - VERSION OPTIMISÉE + VISUALISATIONS
# ==========================================================
# Requirements:
# pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python imbalanced-learn albumentations seaborn

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from mpl_toolkits.mplot3d import Axes3D
import albumentations as A
from albumentations.pytorch import ToTensorV2


# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

ckpt_e4e = os.path.join('encoder4editing','pretrained_models','e4e_ffhq_encode.pt')
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimised"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True


# ----------------------------------------------------------
# CHARGEMENT MODELE
# ----------------------------------------------------------
def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()

        def encode(img_bgr):
            img_rgb = img_bgr[:, :, ::-1]
            tf = T.Compose([T.ToPILImage(), T.Resize((256,256)), T.ToTensor()])
            t = tf(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
                return out.cpu().numpy().reshape(-1)
        print("e4e loaded successfully.")
        return encode
    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None

def load_resnet_fallback():
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Identity()
    model.to(device).eval()

    def encode(img_bgr):
        img_rgb = img_bgr[:, :, ::-1]
        t = transform_resnet(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            return model(t).cpu().numpy().reshape(-1)
    return encode


transform_resnet = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

e4e_model = None
if use_gan_inversion and os.path.exists(ckpt_e4e):
    e4e_model = try_load_e4e(ckpt_e4e)

encoder_fn = e4e_model if e4e_model is not None else load_resnet_fallback()
print("Using encoder:", "e4e" if e4e_model else "ResNet50")


# ----------------------------------------------------------
# LISTE DES IMAGES
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    all_files = []
    for root, dirs, files in os.walk(folder):
        for f in files:
            if f.lower().endswith(exts):
                all_files.append(os.path.join(root, f))
    return all_files

imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)
print("Found REAL:", len(imgs_real))
print("Found FAKE:", len(imgs_fake))


# ----------------------------------------------------------
# AUGMENTATION DATA REAL
# ----------------------------------------------------------
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
])

def augment_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    augmented = augment(image=img)['image']
    return cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)


# ----------------------------------------------------------
# EXTRACTION DES LATENTS
# ----------------------------------------------------------
def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError("Unable to read image " + p)
    return img

X_list = []
y_list = []

for p in tqdm(imgs_real, desc="Encoding REAL"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(0)
        img_aug = augment_image(img)
        X_list.append(encoder_fn(img_aug))
        y_list.append(0)
    except Exception as e:
        print("Error:", p, e)

for p in tqdm(imgs_fake, desc="Encoding FAKE"):
    try:
        img = safe_imread(p)
        X_list.append(encoder_fn(img))
        y_list.append(1)
    except Exception as e:
        print("Error:", p, e)

X = np.array(X_list)
y = np.array(y_list)
print("Latents shape:", X.shape)

np.save(os.path.join(output_dir, "latent_vectors.npy"), X)
np.save(os.path.join(output_dir, "labels.npy"), y)


# ----------------------------------------------------------
# REDUCTION DE DIMENSION
# ----------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=512, random_state=42)
X_reduced = pca.fit_transform(X_scaled)

joblib.dump(pca, os.path.join(output_dir, "pca.joblib"))


# ----------------------------------------------------------
# VISUALISATION 3D_features.png
# ----------------------------------------------------------
pca_3d = PCA(n_components=3)
X_3d = pca_3d.fit_transform(X_scaled)

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_3d[:,0], X_3d[:,1], X_3d[:,2], c=y, cmap='coolwarm', alpha=0.7)
ax.set_title("3D Feature Space (PCA 3D)")
plt.colorbar(scatter)
plt.savefig(os.path.join(output_dir, "3D_features.png"))
plt.close()


# ----------------------------------------------------------
# SMOTE
# ----------------------------------------------------------
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_reduced, y)
print("Dataset après SMOTE :", np.bincount(y_res))


# ----------------------------------------------------------
# TRAIN / TEST SPLIT
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, stratify=y_res, random_state=42
)


# ----------------------------------------------------------
# MODELES
# ----------------------------------------------------------
models_results = {}

svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)
models_results["svm"] = float(accuracy_score(y_test, pred_svm))

rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
models_results["rf"] = float(accuracy_score(y_test, pred_rf))

mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700, random_state=42)
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)
models_results["mlp"] = float(accuracy_score(y_test, pred_mlp))

ratio = np.sum(y_res==0)/np.sum(y_res==1)
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    eval_metric="logloss",
    scale_pos_weight=ratio,
    random_state=42
)
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
models_results["xgb"] = float(accuracy_score(y_test, pred_xgb))


# ----------------------------------------------------------
# MODEL FINAL : VOTING ENSEMBLE
# ----------------------------------------------------------
voting = VotingClassifier(
    estimators=[('rf', rf), ('mlp', mlp), ('xgb', xgb)],
    voting='soft'
)
voting.fit(X_train, y_train)
pred_voting = voting.predict(X_test)

models_results["voting"] = float(accuracy_score(y_test, pred_voting))


# ----------------------------------------------------------
# CONFUSION MATRIX
# ----------------------------------------------------------
cm = confusion_matrix(y_test, pred_voting)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
            xticklabels=["REAL","FAKE"],
            yticklabels=["REAL","FAKE"])
plt.title("Confusion Matrix - Final Model")
plt.savefig(os.path.join(output_dir, "confusion_matrix_final.png"))
plt.close()


# ----------------------------------------------------------
# LEARNING CURVE
# ----------------------------------------------------------
train_sizes, train_scores, test_scores = learning_curve(
    voting, X_train, y_train, cv=5,
    train_sizes=np.linspace(0.1,1.0,10),
    scoring='accuracy',
    shuffle=True,
    random_state=42
)

train_mean = train_scores.mean(axis=1)
test_mean = test_scores.mean(axis=1)

plt.figure(figsize=(8,6))
plt.plot(train_sizes, train_mean, label="Training Score")
plt.plot(train_sizes, test_mean, label="Validation Score")
plt.xlabel("Training Samples")
plt.ylabel("Accuracy")
plt.title("Learning Curve - Final Model")
plt.legend()
plt.grid()
plt.savefig(os.path.join(output_dir, "learning_curve.png"))
plt.close()


# ----------------------------------------------------------
# SAVE MODELS
# ----------------------------------------------------------
joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))
joblib.dump(svm, os.path.join(output_dir, "model_svm.joblib"))
joblib.dump(rf, os.path.join(output_dir, "model_rf.joblib"))
joblib.dump(mlp, os.path.join(output_dir, "model_mlp.joblib"))
joblib.dump(xgb, os.path.join(output_dir, "model_xgb.joblib"))
joblib.dump(voting, os.path.join(output_dir, "model_voting.joblib"))

with open(os.path.join(output_dir, "results_models.json"), "w") as f:
    json.dump(models_results, f, indent=2)


print("\nAccuracy des modèles :")
for m, a in models_results.items():
    print(f" - {m}: {a:.4f}")

print("\nPipeline terminé ! Résultats dans :", output_dir)


Device: cuda


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_15792\1550336598.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

e4e loaded successfully.
Using encoder: e4e
Found REAL: 790
Found FAKE: 3975


Encoding FAKE: 100%|██████████| 3975/3975 [07:05<00:00,  9.34it/s]


Latents shape: (5555, 9216)


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Dataset après SMOTE : [3975 3975]

Accuracy des modèles :
 - svm: 0.8214
 - rf: 0.8428
 - mlp: 0.9252
 - xgb: 0.8566
 - voting: 0.9170

Pipeline terminé ! Résultats dans : C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimised


In [3]:
# ==========================================================
# Deepfake detection pipeline - VERSION OPTIMISÉE + 3D VIS + SAFE DISTANCES
# ==========================================================
# Requirements:
# pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python imbalanced-learn albumentations seaborn

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from mpl_toolkits.mplot3d import Axes3D
import albumentations as A

# Additional analyses
from sklearn.manifold import TSNE
import umap.umap_ as umap
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

# paths (tes frames)
path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\artifacts\frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\artifacts\frames\fake"

ckpt_e4e = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimiseddd"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True

# ----------------------------------------------------------
# MODEL LOADING (e4e or ResNet fallback)
# ----------------------------------------------------------
def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()

        # single-image encode (keeps original behavior)
        tf_e4e = T.Compose([T.ToPILImage(), T.Resize((256,256)), T.ToTensor()])

        def encode(img_bgr):
            img_rgb = img_bgr[:, :, ::-1]
            t = tf_e4e(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
            return out.cpu().numpy().reshape(-1)

        # batch encoder wrapper (list of BGR images)
        def encode_batch(imgs_bgr_list):
            tensors = []
            for img_bgr in imgs_bgr_list:
                img_rgb = img_bgr[:, :, ::-1]
                tensors.append(tf_e4e(img_rgb))
            batch_t = torch.stack(tensors).to(device)
            with torch.no_grad():
                out = net.encoder(batch_t)
            return out.cpu().numpy()

        print("e4e loaded successfully.")
        return encode, encode_batch

    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None, None

def load_resnet_fallback():
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Identity()
    model.to(device).eval()

    tf_res = T.Compose([
        T.ToPILImage(), T.Resize((224,224)), T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])

    def encode(img_bgr):
        img_rgb = img_bgr[:, :, ::-1]
        t = tf_res(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(t)
        return out.cpu().numpy().reshape(-1)

    def encode_batch(imgs_bgr_list):
        tensors = []
        for img_bgr in imgs_bgr_list:
            img_rgb = img_bgr[:, :, ::-1]
            tensors.append(tf_res(img_rgb))
        batch_t = torch.stack(tensors).to(device)
        with torch.no_grad():
            out = model(batch_t)
        return out.cpu().numpy()

    return encode, encode_batch

# instantiate encoders
encode_single = None
encode_batch = None
if use_gan_inversion and os.path.exists(ckpt_e4e):
    encode_single, encode_batch = try_load_e4e(ckpt_e4e)

if encode_single is None:
    encode_single, encode_batch = load_resnet_fallback()

print("Using encoder:", "e4e" if os.path.exists(ckpt_e4e) and encode_single is not None else "ResNet50")

# ----------------------------------------------------------
# IO helpers
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    files = []
    for root, _, names in os.walk(folder):
        for f in names:
            if f.lower().endswith(exts):
                files.append(os.path.join(root, f))
    return sorted(files)

def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError("Unable to read image " + p)
    return img

# augmentation (albumentations)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
])

def augment_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aug = augment(image=img_rgb)['image']
    return cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)

# ----------------------------------------------------------
# LIST IMAGES
# ----------------------------------------------------------
imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)
print("Found REAL:", len(imgs_real), "FAKE:", len(imgs_fake))

# ----------------------------------------------------------
# ENCODING (BATCH = 8) with augmentation for REAL
# ----------------------------------------------------------
batch_size = 16

X_list = []
y_list = []
paths_list = []

def process_paths_in_batches(paths, label, augment_real=False, batch_size=8):
    buff_imgs = []
    buff_paths = []
    for p in tqdm(paths):
        try:
            img = safe_imread(p)
            buff_imgs.append(img)
            buff_paths.append(p)

            # if augmentation requested (only for REAL), we append an augmented image to buffer
            if augment_real:
                img_aug = augment_image(img)
                buff_imgs.append(img_aug)
                buff_paths.append(p + "||aug")

            # when buffer reaches batch_size (or more), encode them in batch
            while len(buff_imgs) >= batch_size:
                batch_imgs = buff_imgs[:batch_size]
                batch_paths = buff_paths[:batch_size]
                # encode_batch returns (n, latent_dim)
                latents = encode_batch(batch_imgs)
                for latent, path in zip(latents, batch_paths):
                    X_list.append(latent)
                    y_list.append(label)
                    paths_list.append(path)
                # pop used
                buff_imgs = buff_imgs[batch_size:]
                buff_paths = buff_paths[batch_size:]
        except Exception as e:
            print("Error reading/encoding:", p, e)

    # flush remainder
    if len(buff_imgs) > 0:
        latents = encode_batch(buff_imgs)
        for latent, path in zip(latents, buff_paths):
            X_list.append(latent)
            y_list.append(label)
            paths_list.append(path)

# run encoding
print("Encoding REAL (with augmentation)...")
process_paths_in_batches(imgs_real, label=0, augment_real=True, batch_size=batch_size)

print("Encoding FAKE ...")
process_paths_in_batches(imgs_fake, label=1, augment_real=False, batch_size=batch_size)
# ----------------------------------------------------------
# convert to arrays
X = np.array(X_list)
y = np.array(y_list)
print("Latents shape:", X.shape)

# Flatten / average latents if 3D
if X.ndim == 3:
    X = X.mean(axis=1)
    print("Latents averaged over layers -> new shape:", X.shape)

np.save(os.path.join(output_dir, "latent_vectors.npy"), X)
np.save(os.path.join(output_dir, "labels.npy"), y)

# ----------------------------------------------------------
# SAFE STATISTICS: mean/var (dimension-aggregated) + sampled distances
# ----------------------------------------------------------
if len(np.unique(y)) == 2:
    X_real = X[y == 0]
    X_fake = X[y == 1]
else:
    X_real = X
    X_fake = np.zeros((0, X.shape[1]))

# compute per-class mean/var (dimension-wise)
mean_real_dimmean = float(np.mean(X_real.mean(axis=0))) if X_real.shape[0] > 0 else None
var_real_dimmean = float(np.mean(X_real.var(axis=0))) if X_real.shape[0] > 0 else None
mean_fake_dimmean = float(np.mean(X_fake.mean(axis=0))) if X_fake.shape[0] > 0 else None
var_fake_dimmean = float(np.mean(X_fake.var(axis=0))) if X_fake.shape[0] > 0 else None

# distances: sample to avoid huge memory
MAX_SAMPLES = 1500
def sample_array(arr, max_n):
    n = len(arr)
    if n <= max_n:
        return arr
    idx = np.random.choice(n, max_n, replace=False)
    return arr[idx]

Xr = sample_array(X_real, MAX_SAMPLES)
Xf = sample_array(X_fake, MAX_SAMPLES)

intra_real = None
intra_fake = None
inter_mean = None

if Xr.shape[0] > 1:
    dr = cdist(Xr, Xr, metric='euclidean')
    intra_real = float(np.mean(dr[np.triu_indices_from(dr, 1)]))
if Xf.shape[0] > 1:
    df = cdist(Xf, Xf, metric='euclidean')
    intra_fake = float(np.mean(df[np.triu_indices_from(df, 1)]))
if Xr.shape[0] > 0 and Xf.shape[0] > 0:
    di = cdist(Xr, Xf, metric='euclidean')
    inter_mean = float(np.mean(di))

analysis_compact = {
    "real_count": int(len(X_real)),
    "fake_count": int(len(X_fake)),
    "sampled_real": int(len(Xr)),
    "sampled_fake": int(len(Xf)),
    "mean_of_mean_real": mean_real_dimmean,
    "var_of_mean_real": var_real_dimmean,
    "mean_of_mean_fake": mean_fake_dimmean,
    "var_of_mean_fake": var_fake_dimmean,
    "intra_real_distance_sampled": intra_real,
    "intra_fake_distance_sampled": intra_fake,
    "inter_mean_distance_sampled": inter_mean
}

with open(os.path.join(output_dir, "latent_analysis_extended.json"), "w") as f:
    json.dump(analysis_compact, f, indent=2)

print("Saved latent_analysis_extended.json (compact stats)")


# ----------------------------------------------------------
# STANDARDIZE and INCREMENTAL PCA (memory-friendly)
# ----------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # X is already in memory as np.array
joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))

# Incremental PCA to 512 components
n_components_big = 512
ipca = IncrementalPCA(n_components=n_components_big, batch_size=1024)
for start in range(0, X_scaled.shape[0], 1024):
    ipca.partial_fit(X_scaled[start:start+1024])
X_reduced = ipca.transform(X_scaled)
joblib.dump(ipca, os.path.join(output_dir, "incremental_pca_512.joblib"))
print("Incremental PCA -> shape:", X_reduced.shape)

# ----------------------------------------------------------
# PCA 3D visualization (from X_reduced -> PCA 3 components)
# ----------------------------------------------------------
pca3 = PCA(n_components=3, random_state=42)
X_3d = pca3.fit_transform(X_reduced)  # shape (N,3)

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_3d[:,0], X_3d[:,1], X_3d[:,2], c=y, cmap='coolwarm', s=4, alpha=0.7)
ax.set_title("3D Feature Space (PCA 3D) on reduced features")
ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.set_zlabel("PC3")
plt.colorbar(scatter, ax=ax, fraction=0.02, pad=0.1)
plt.savefig(os.path.join(output_dir, "3D_features.png"), dpi=150)
plt.close()
print("Saved 3D_features.png")

# ----------------------------------------------------------
# PCA 2D (for visualization)
# ----------------------------------------------------------
pca2 = PCA(n_components=2, random_state=42)
X_pca2 = pca2.fit_transform(X_reduced)
plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=y, cmap='coolwarm', s=4, alpha=0.7)
plt.title("PCA 2D (on reduced features)")
plt.savefig(os.path.join(output_dir, "pca_2d.png"))
plt.close()

# ----------------------------------------------------------
# t-SNE and UMAP (on a sample for performance)
# ----------------------------------------------------------
TS_NEIGHBORS = 3000  # sample size for TSNE/UMAP
if X_reduced.shape[0] > TS_NEIGHBORS:
    idx_sample = np.random.choice(X_reduced.shape[0], TS_NEIGHBORS, replace=False)
    X_vis = X_reduced[idx_sample]
    y_vis = y[idx_sample]
else:
    X_vis = X_reduced
    y_vis = y

# t-SNE (2D)
tsne = TSNE(n_components=2, perplexity=40, n_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(X_vis)
plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y_vis, cmap='coolwarm', s=4, alpha=0.7)
plt.title("t-SNE (2D) on a sample")
plt.savefig(os.path.join(output_dir, "tsne_2d.png"))
plt.close()

# UMAP (2D)
reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_vis)
plt.figure(figsize=(8,6))
plt.scatter(X_umap[:,0], X_umap[:,1], c=y_vis, cmap='coolwarm', s=4, alpha=0.7)
plt.title("UMAP (2D) on a sample")
plt.savefig(os.path.join(output_dir, "umap_2d.png"))
plt.close()

# ----------------------------------------------------------
# CLUSTERING (KMeans 2 clusters) on a sample and silhouette
# ----------------------------------------------------------
km_sample = X_vis  # use sampled subset
kmeans = KMeans(n_clusters=2, random_state=42)
cluster_labels = kmeans.fit_predict(km_sample)
from sklearn.metrics import silhouette_score
sil = silhouette_score(km_sample, cluster_labels) if len(km_sample) > 1 else None

plt.figure(figsize=(8,6))
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=cluster_labels.repeat(int(np.ceil(len(X_pca2)/len(cluster_labels))))[:len(X_pca2)], cmap='tab10', s=4, alpha=0.7)
plt.title(f"KMeans clusters (sample) — silhouette={sil}")
plt.savefig(os.path.join(output_dir, "kmeans_pca2d.png"))
plt.close()

# ----------------------------------------------------------
# SMOTE balancing on X_reduced
# ----------------------------------------------------------
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_reduced, y)
print("After SMOTE class counts:", np.bincount(y_res))

# ----------------------------------------------------------
# TRAIN / TEST SPLIT
# ----------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, stratify=y_res, random_state=42)

# ----------------------------------------------------------
# MODELS (SVM, RF, MLP, XGB) and Voting Ensemble
# ----------------------------------------------------------
models_results = {}

# SVM
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)
models_results["svm"] = float(accuracy_score(y_test, pred_svm))

# Random Forest
rf = RandomForestClassifier(n_estimators=500, random_state=42)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
models_results["rf"] = float(accuracy_score(y_test, pred_rf))

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700, random_state=42)
mlp.fit(X_train, y_train)
pred_mlp = mlp.predict(X_test)
models_results["mlp"] = float(accuracy_score(y_test, pred_mlp))

# XGBoost
ratio = np.sum(y_res==0) / np.sum(y_res==1)
xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, eval_metric="logloss", scale_pos_weight=ratio, random_state=42, use_label_encoder=False)
xgb.fit(X_train, y_train)
pred_xgb = xgb.predict(X_test)
models_results["xgb"] = float(accuracy_score(y_test, pred_xgb))

# Voting
voting = VotingClassifier(estimators=[('rf', rf), ('mlp', mlp), ('xgb', xgb)], voting='soft')
voting.fit(X_train, y_train)
pred_voting = voting.predict(X_test)
models_results["voting"] = float(accuracy_score(y_test, pred_voting))

# ----------------------------------------------------------
# Confusion matrix (final voting model)
# ----------------------------------------------------------
cm = confusion_matrix(y_test, pred_voting)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=["REAL","FAKE"], yticklabels=["REAL","FAKE"])
plt.title("Confusion Matrix - Voting")
plt.savefig(os.path.join(output_dir, "confusion_matrix_final.png"))
plt.close()

# ----------------------------------------------------------
# Learning curve (final)
# ----------------------------------------------------------
train_sizes, train_scores, test_scores = learning_curve(voting, X_train, y_train, cv=5,
                                                       train_sizes=np.linspace(0.1,1.0,10),
                                                       scoring='accuracy', shuffle=True, random_state=42)
plt.figure(figsize=(8,6))
plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
plt.xlabel("Training samples")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.title("Learning Curve - Voting")
plt.savefig(os.path.join(output_dir, "learning_curve.png"))
plt.close()

# ----------------------------------------------------------
# SAVE MODELS & RESULTS
# ----------------------------------------------------------
joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))
joblib.dump(ipca, os.path.join(output_dir, "incremental_pca_512.joblib"))
joblib.dump(svm, os.path.join(output_dir, "model_svm.joblib"))
joblib.dump(rf, os.path.join(output_dir, "model_rf.joblib"))
joblib.dump(mlp, os.path.join(output_dir, "model_mlp.joblib"))
joblib.dump(xgb, os.path.join(output_dir, "model_xgb.joblib"))
joblib.dump(voting, os.path.join(output_dir, "model_voting.joblib"))

with open(os.path.join(output_dir, "results_models.json"), "w") as f:
    json.dump(models_results, f, indent=2)

with open(os.path.join(output_dir, "latent_analysis_extended.json"), "w") as f:
    json.dump(analysis_compact, f, indent=2)

print("\nAccuracy des modèles :")
for k,v in models_results.items():
    print(f" - {k}: {v:.4f}")

print("\nPipeline terminé ! Résultats dans :", output_dir)


Device: cuda


C:\Users\EliteLaptop\AppData\Local\Temp\ipykernel_17152\429933128.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
e4e loaded successfully.
Using encoder: e4e
Found REAL: 12361 FAKE: 62877
Encoding REAL (with augmentation)...


100%|██████████| 12361/12361 [1:03:04<00:00,  3.27it/s]


Encoding FAKE ...


100%|██████████| 62877/62877 [2:26:35<00:00,  7.15it/s]  


Latents shape: (87599, 18, 512)
Latents averaged over layers -> new shape: (87599, 512)
Saved latent_analysis_extended.json (compact stats)
Incremental PCA -> shape: (87599, 512)
Saved 3D_features.png


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


After SMOTE class counts: [62877 62877]


KeyboardInterrupt: 

In [5]:
# # ==========================================================
# # Deepfake detection pipeline - VERSION OPTIMISÉE + 3D VIS + SAFE DISTANCES
# # ==========================================================
# # Requirements:
# # pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python imbalanced-learn albumentations seaborn

# import os
# import cv2
# import numpy as np
# import torch
# import torchvision.transforms as T
# import torchvision.models as models
# from tqdm import tqdm
# import matplotlib.pyplot as plt
# import seaborn as sns
# import joblib
# import json

# from sklearn.decomposition import PCA, IncrementalPCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split, learning_curve
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix
# from xgboost import XGBClassifier
# from imblearn.over_sampling import SMOTE
# from mpl_toolkits.mplot3d import Axes3D
# import albumentations as A

# # Additional analyses
# from sklearn.manifold import TSNE
# import umap.umap_ as umap
# from scipy.spatial.distance import cdist
# from sklearn.cluster import KMeans

# # ----------------------------------------------------------
# # CONFIGURATION
# # ----------------------------------------------------------
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print("Device:", device)

# # paths (tes frames)
# path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
# path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

# ckpt_e4e = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
# output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimisedddd"
# os.makedirs(output_dir, exist_ok=True)

# use_gan_inversion = True

# # ----------------------------------------------------------
# # MODEL LOADING (e4e or ResNet fallback)
# # ----------------------------------------------------------
# def try_load_e4e(ckpt_path):
#     try:
#         from encoder4editing.models.psp import pSp
#         ckpt = torch.load(ckpt_path, map_location='cpu')
#         opts = ckpt['opts']
#         opts['checkpoint_path'] = ckpt_path
#         opts = type('Options', (), opts)()
#         net = pSp(opts).to(device).eval()

#         # single-image encode (keeps original behavior)
#         tf_e4e = T.Compose([T.ToPILImage(), T.Resize((256,256)), T.ToTensor()])

#         def encode(img_bgr):
#             img_rgb = img_bgr[:, :, ::-1]
#             t = tf_e4e(img_rgb).unsqueeze(0).to(device)
#             with torch.no_grad():
#                 out = net.encoder(t)
#             return out.cpu().numpy().reshape(-1)

#         # batch encoder wrapper (list of BGR images)
#         def encode_batch(imgs_bgr_list):
#             tensors = []
#             for img_bgr in imgs_bgr_list:
#                 img_rgb = img_bgr[:, :, ::-1]
#                 tensors.append(tf_e4e(img_rgb))
#             batch_t = torch.stack(tensors).to(device)
#             with torch.no_grad():
#                 out = net.encoder(batch_t)
#             return out.cpu().numpy()

#         print("e4e loaded successfully.")
#         return encode, encode_batch

#     except Exception as e:
#         print("Erreur chargement e4e :", e)
#         return None, None

# def load_resnet_fallback():
#     model = models.resnet50(pretrained=True)
#     model.fc = torch.nn.Identity()
#     model.to(device).eval()

#     tf_res = T.Compose([
#         T.ToPILImage(), T.Resize((224,224)), T.ToTensor(),
#         T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
#     ])

#     def encode(img_bgr):
#         img_rgb = img_bgr[:, :, ::-1]
#         t = tf_res(img_rgb).unsqueeze(0).to(device)
#         with torch.no_grad():
#             out = model(t)
#         return out.cpu().numpy().reshape(-1)

#     def encode_batch(imgs_bgr_list):
#         tensors = []
#         for img_bgr in imgs_bgr_list:
#             img_rgb = img_bgr[:, :, ::-1]
#             tensors.append(tf_res(img_rgb))
#         batch_t = torch.stack(tensors).to(device)
#         with torch.no_grad():
#             out = model(batch_t)
#         return out.cpu().numpy()

#     return encode, encode_batch

# # instantiate encoders
# encode_single = None
# encode_batch = None
# if use_gan_inversion and os.path.exists(ckpt_e4e):
#     encode_single, encode_batch = try_load_e4e(ckpt_e4e)

# if encode_single is None:
#     encode_single, encode_batch = load_resnet_fallback()

# print("Using encoder:", "e4e" if os.path.exists(ckpt_e4e) and encode_single is not None else "ResNet50")

# # ----------------------------------------------------------
# # IO helpers
# # ----------------------------------------------------------
# def list_images(folder):
#     exts = ('.jpg','.jpeg','.png','.bmp')
#     files = []
#     for root, _, names in os.walk(folder):
#         for f in names:
#             if f.lower().endswith(exts):
#                 files.append(os.path.join(root, f))
#     return sorted(files)

# def safe_imread(p):
#     img = cv2.imread(p)
#     if img is None:
#         raise ValueError("Unable to read image " + p)
#     return img

# # augmentation (albumentations)
# augment = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.RandomBrightnessContrast(p=0.3),
#     A.Rotate(limit=15, p=0.3),
# ])

# def augment_image(img):
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     aug = augment(image=img_rgb)['image']
#     return cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)

# # ----------------------------------------------------------
# # LIST IMAGES
# # ----------------------------------------------------------
# imgs_real = list_images(path_real)
# imgs_fake = list_images(path_fake)
# print("Found REAL:", len(imgs_real), "FAKE:", len(imgs_fake))

# # ----------------------------------------------------------
# # ENCODING (BATCH = 8) with augmentation for REAL
# # ----------------------------------------------------------
# batch_size = 16

# X_list = []
# y_list = []
# paths_list = []

# def process_paths_in_batches(paths, label, augment_real=False, batch_size=8):
#     buff_imgs = []
#     buff_paths = []
#     for p in tqdm(paths):
#         try:
#             img = safe_imread(p)
#             buff_imgs.append(img)
#             buff_paths.append(p)

#             # if augmentation requested (only for REAL), we append an augmented image to buffer
#             if augment_real:
#                 img_aug = augment_image(img)
#                 buff_imgs.append(img_aug)
#                 buff_paths.append(p + "||aug")

#             # when buffer reaches batch_size (or more), encode them in batch
#             while len(buff_imgs) >= batch_size:
#                 batch_imgs = buff_imgs[:batch_size]
#                 batch_paths = buff_paths[:batch_size]
#                 # encode_batch returns (n, latent_dim)
#                 latents = encode_batch(batch_imgs)
#                 for latent, path in zip(latents, batch_paths):
#                     X_list.append(latent)
#                     y_list.append(label)
#                     paths_list.append(path)
#                 # pop used
#                 buff_imgs = buff_imgs[batch_size:]
#                 buff_paths = buff_paths[batch_size:]
#         except Exception as e:
#             print("Error reading/encoding:", p, e)

#     # flush remainder
#     if len(buff_imgs) > 0:
#         latents = encode_batch(buff_imgs)
#         for latent, path in zip(latents, buff_paths):
#             X_list.append(latent)
#             y_list.append(label)
#             paths_list.append(path)

# # run encoding
# print("Encoding REAL (with augmentation)...")
# process_paths_in_batches(imgs_real, label=0, augment_real=True, batch_size=batch_size)

# print("Encoding FAKE ...")
# process_paths_in_batches(imgs_fake, label=1, augment_real=False, batch_size=batch_size)
# # ----------------------------------------------------------
# # convert to arrays
# X = np.array(X_list)
# y = np.array(y_list)
# print("Latents shape:", X.shape)

# # Flatten / average latents if 3D
# if X.ndim == 3:
#     X = X.mean(axis=1)
#     print("Latents averaged over layers -> new shape:", X.shape)

# np.save(os.path.join(output_dir, "latent_vectors.npy"), X)
# np.save(os.path.join(output_dir, "labels.npy"), y)

# # ----------------------------------------------------------
# # SAFE STATISTICS: mean/var (dimension-aggregated) + sampled distances
# # ----------------------------------------------------------
# if len(np.unique(y)) == 2:
#     X_real = X[y == 0]
#     X_fake = X[y == 1]
# else:
#     X_real = X
#     X_fake = np.zeros((0, X.shape[1]))

# # compute per-class mean/var (dimension-wise)
# mean_real_dimmean = float(np.mean(X_real.mean(axis=0))) if X_real.shape[0] > 0 else None
# var_real_dimmean = float(np.mean(X_real.var(axis=0))) if X_real.shape[0] > 0 else None
# mean_fake_dimmean = float(np.mean(X_fake.mean(axis=0))) if X_fake.shape[0] > 0 else None
# var_fake_dimmean = float(np.mean(X_fake.var(axis=0))) if X_fake.shape[0] > 0 else None

# # distances: sample to avoid huge memory
# MAX_SAMPLES = 1500
# def sample_array(arr, max_n):
#     n = len(arr)
#     if n <= max_n:
#         return arr
#     idx = np.random.choice(n, max_n, replace=False)
#     return arr[idx]

# Xr = sample_array(X_real, MAX_SAMPLES)
# Xf = sample_array(X_fake, MAX_SAMPLES)

# intra_real = None
# intra_fake = None
# inter_mean = None

# if Xr.shape[0] > 1:
#     dr = cdist(Xr, Xr, metric='euclidean')
#     intra_real = float(np.mean(dr[np.triu_indices_from(dr, 1)]))
# if Xf.shape[0] > 1:
#     df = cdist(Xf, Xf, metric='euclidean')
#     intra_fake = float(np.mean(df[np.triu_indices_from(df, 1)]))
# if Xr.shape[0] > 0 and Xf.shape[0] > 0:
#     di = cdist(Xr, Xf, metric='euclidean')
#     inter_mean = float(np.mean(di))

# analysis_compact = {
#     "real_count": int(len(X_real)),
#     "fake_count": int(len(X_fake)),
#     "sampled_real": int(len(Xr)),
#     "sampled_fake": int(len(Xf)),
#     "mean_of_mean_real": mean_real_dimmean,
#     "var_of_mean_real": var_real_dimmean,
#     "mean_of_mean_fake": mean_fake_dimmean,
#     "var_of_mean_fake": var_fake_dimmean,
#     "intra_real_distance_sampled": intra_real,
#     "intra_fake_distance_sampled": intra_fake,
#     "inter_mean_distance_sampled": inter_mean
# }

# with open(os.path.join(output_dir, "latent_analysis_extended.json"), "w") as f:
#     json.dump(analysis_compact, f, indent=2)

# print("Saved latent_analysis_extended.json (compact stats)")


# # ----------------------------------------------------------
# # STANDARDIZE and INCREMENTAL PCA (memory-friendly)
# # ----------------------------------------------------------
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)  # X is already in memory as np.array
# joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))

# # Incremental PCA to 512 components
# n_components_big = 512
# ipca = IncrementalPCA(n_components=n_components_big, batch_size=1024)
# for start in range(0, X_scaled.shape[0], 1024):
#     ipca.partial_fit(X_scaled[start:start+1024])
# X_reduced = ipca.transform(X_scaled)
# joblib.dump(ipca, os.path.join(output_dir, "incremental_pca_512.joblib"))
# print("Incremental PCA -> shape:", X_reduced.shape)

# # ----------------------------------------------------------
# # PCA 3D visualization (from X_reduced -> PCA 3 components)
# # ----------------------------------------------------------
# pca3 = PCA(n_components=3, random_state=42)
# X_3d = pca3.fit_transform(X_reduced)  # shape (N,3)

# fig = plt.figure(figsize=(10,8))
# ax = fig.add_subplot(111, projection='3d')
# scatter = ax.scatter(X_3d[:,0], X_3d[:,1], X_3d[:,2], c=y, cmap='coolwarm', s=4, alpha=0.7)
# ax.set_title("3D Feature Space (PCA 3D) on reduced features")
# ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.set_zlabel("PC3")
# plt.colorbar(scatter, ax=ax, fraction=0.02, pad=0.1)
# plt.savefig(os.path.join(output_dir, "3D_features.png"), dpi=150)
# plt.close()
# print("Saved 3D_features.png")

# # ----------------------------------------------------------
# # PCA 2D (for visualization)
# # ----------------------------------------------------------
# pca2 = PCA(n_components=2, random_state=42)
# X_pca2 = pca2.fit_transform(X_reduced)
# plt.figure(figsize=(8,6))
# plt.scatter(X_pca2[:,0], X_pca2[:,1], c=y, cmap='coolwarm', s=4, alpha=0.7)
# plt.title("PCA 2D (on reduced features)")
# plt.savefig(os.path.join(output_dir, "pca_2d.png"))
# plt.close()

# # ----------------------------------------------------------
# # t-SNE and UMAP (on a sample for performance)
# # ----------------------------------------------------------
# TS_NEIGHBORS = 3000  # sample size for TSNE/UMAP
# if X_reduced.shape[0] > TS_NEIGHBORS:
#     idx_sample = np.random.choice(X_reduced.shape[0], TS_NEIGHBORS, replace=False)
#     X_vis = X_reduced[idx_sample]
#     y_vis = y[idx_sample]
# else:
#     X_vis = X_reduced
#     y_vis = y

# # t-SNE (2D)
# tsne = TSNE(n_components=2, perplexity=40, n_iter=1000, random_state=42)
# X_tsne = tsne.fit_transform(X_vis)
# plt.figure(figsize=(8,6))
# plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y_vis, cmap='coolwarm', s=4, alpha=0.7)
# plt.title("t-SNE (2D) on a sample")
# plt.savefig(os.path.join(output_dir, "tsne_2d.png"))
# plt.close()

# # UMAP (2D)
# reducer = umap.UMAP(n_components=2, random_state=42)
# X_umap = reducer.fit_transform(X_vis)
# plt.figure(figsize=(8,6))
# plt.scatter(X_umap[:,0], X_umap[:,1], c=y_vis, cmap='coolwarm', s=4, alpha=0.7)
# plt.title("UMAP (2D) on a sample")
# plt.savefig(os.path.join(output_dir, "umap_2d.png"))
# plt.close()

# # ----------------------------------------------------------
# # CLUSTERING (KMeans 2 clusters) on a sample and silhouette
# # ----------------------------------------------------------
# km_sample = X_vis  # use sampled subset
# kmeans = KMeans(n_clusters=2, random_state=42)
# cluster_labels = kmeans.fit_predict(km_sample)
# from sklearn.metrics import silhouette_score
# sil = silhouette_score(km_sample, cluster_labels) if len(km_sample) > 1 else None

# plt.figure(figsize=(8,6))
# plt.scatter(X_pca2[:,0], X_pca2[:,1], c=cluster_labels.repeat(int(np.ceil(len(X_pca2)/len(cluster_labels))))[:len(X_pca2)], cmap='tab10', s=4, alpha=0.7)
# plt.title(f"KMeans clusters (sample) — silhouette={sil}")
# plt.savefig(os.path.join(output_dir, "kmeans_pca2d.png"))
# plt.close()

# # ----------------------------------------------------------
# # SMOTE balancing on X_reduced
# # ----------------------------------------------------------
# smote = SMOTE(random_state=42)
# X_res, y_res = smote.fit_resample(X_reduced, y)
# print("After SMOTE class counts:", np.bincount(y_res))

# # ----------------------------------------------------------
# # TRAIN / TEST SPLIT
# # ----------------------------------------------------------
# X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, stratify=y_res, random_state=42)

# # ----------------------------------------------------------
# # MODELS (SVM, RF, MLP, XGB) and Voting Ensemble
# # ----------------------------------------------------------
# models_results = {}

# # SVM
# svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
# svm.fit(X_train, y_train)
# pred_svm = svm.predict(X_test)
# models_results["svm"] = float(accuracy_score(y_test, pred_svm))

# # Random Forest
# rf = RandomForestClassifier(n_estimators=500, random_state=42)
# rf.fit(X_train, y_train)
# pred_rf = rf.predict(X_test)
# models_results["rf"] = float(accuracy_score(y_test, pred_rf))

# # MLP
# mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700, random_state=42)
# mlp.fit(X_train, y_train)
# pred_mlp = mlp.predict(X_test)
# models_results["mlp"] = float(accuracy_score(y_test, pred_mlp))

# # XGBoost
# ratio = np.sum(y_res==0) / np.sum(y_res==1)
# xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, eval_metric="logloss", scale_pos_weight=ratio, random_state=42, use_label_encoder=False)
# xgb.fit(X_train, y_train)
# pred_xgb = xgb.predict(X_test)
# models_results["xgb"] = float(accuracy_score(y_test, pred_xgb))

# # Voting
# voting = VotingClassifier(estimators=[('rf', rf), ('mlp', mlp), ('xgb', xgb)], voting='soft')
# voting.fit(X_train, y_train)
# pred_voting = voting.predict(X_test)
# models_results["voting"] = float(accuracy_score(y_test, pred_voting))

# # ----------------------------------------------------------
# # Confusion matrix (final voting model)
# # ----------------------------------------------------------
# cm = confusion_matrix(y_test, pred_voting)
# plt.figure(figsize=(6,5))
# sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=["REAL","FAKE"], yticklabels=["REAL","FAKE"])
# plt.title("Confusion Matrix - Voting")
# plt.savefig(os.path.join(output_dir, "confusion_matrix_final.png"))
# plt.close()

# # ----------------------------------------------------------
# # Learning curve (final)
# # ----------------------------------------------------------
# train_sizes, train_scores, test_scores = learning_curve(voting, X_train, y_train, cv=5,
#                                                        train_sizes=np.linspace(0.1,1.0,10),
#                                                        scoring='accuracy', shuffle=True, random_state=42)
# plt.figure(figsize=(8,6))
# plt.plot(train_sizes, train_scores.mean(axis=1), label='Training Score')
# plt.plot(train_sizes, test_scores.mean(axis=1), label='Validation Score')
# plt.xlabel("Training samples")
# plt.ylabel("Accuracy")
# plt.legend()
# plt.grid()
# plt.title("Learning Curve - Voting")
# plt.savefig(os.path.join(output_dir, "learning_curve.png"))
# plt.close()

# # ----------------------------------------------------------
# # SAVE MODELS & RESULTS
# # ----------------------------------------------------------
# joblib.dump(scaler, os.path.join(output_dir, "scaler.joblib"))
# joblib.dump(ipca, os.path.join(output_dir, "incremental_pca_512.joblib"))
# joblib.dump(svm, os.path.join(output_dir, "model_svm.joblib"))
# joblib.dump(rf, os.path.join(output_dir, "model_rf.joblib"))
# joblib.dump(mlp, os.path.join(output_dir, "model_mlp.joblib"))
# joblib.dump(xgb, os.path.join(output_dir, "model_xgb.joblib"))
# joblib.dump(voting, os.path.join(output_dir, "model_voting.joblib"))

# with open(os.path.join(output_dir, "results_models.json"), "w") as f:
#     json.dump(models_results, f, indent=2)

# with open(os.path.join(output_dir, "latent_analysis_extended.json"), "w") as f:
#     json.dump(analysis_compact, f, indent=2)

# print("\nAccuracy des modèles :")
# for k,v in models_results.items():
#     print(f" - {k}: {v:.4f}")

# print("\nPipeline terminé ! Résultats dans :", output_dir)


# ==========================================================
# Deepfake detection pipeline - VERSION OPTIMISÉE + 3D VIS + SAFE DISTANCES
# ==========================================================
# Requirements:
# pip install torch torchvision timm scikit-learn xgboost umap-learn matplotlib tqdm opencv-python imbalanced-learn albumentations seaborn

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from mpl_toolkits.mplot3d import Axes3D
import albumentations as A

from sklearn.manifold import TSNE
import umap.umap_ as umap
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\extracted_frames\fake"

ckpt_e4e = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimiseddddd"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True

# ----------------------------------------------------------
# MODEL LOADING (e4e or ResNet fallback)
# ----------------------------------------------------------
def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu', weights_only=True)
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()

        tf_e4e = T.Compose([
            T.ToPILImage(),
            T.Resize((256,256)),
            T.ToTensor()
        ])

        def encode(img_bgr):
            img_rgb = img_bgr[:, :, ::-1]
            t = tf_e4e(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
            return out.cpu().numpy().reshape(-1)

        def encode_batch(imgs_bgr_list):
            tensors = []
            for img_bgr in imgs_bgr_list:
                img_rgb = img_bgr[:, :, ::-1]
                tensors.append(tf_e4e(img_rgb))
            batch_t = torch.stack(tensors).to(device)
            with torch.no_grad():
                out = net.encoder(batch_t)
            return out.cpu().numpy()

        print("e4e loaded successfully.")
        return encode, encode_batch

    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None, None

def load_resnet_fallback():
    model = models.resnet50(weights="DEFAULT")
    model.fc = torch.nn.Identity()
    model.to(device).eval()

    tf_res = T.Compose([
        T.ToPILImage(),
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])

    def encode(img_bgr):
        img_rgb = img_bgr[:, :, ::-1]
        t = tf_res(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(t)
        return out.cpu().numpy().reshape(-1)

    def encode_batch(imgs_bgr_list):
        tensors = []
        for img_bgr in imgs_bgr_list:
            img_rgb = img_bgr[:, :, ::-1]
            tensors.append(tf_res(img_rgb))
        batch_t = torch.stack(tensors).to(device)
        with torch.no_grad():
            out = model(batch_t)
        return out.cpu().numpy()

    return encode, encode_batch

encode_single, encode_batch = None, None
if use_gan_inversion and os.path.exists(ckpt_e4e):
    encode_single, encode_batch = try_load_e4e(ckpt_e4e)

if encode_single is None:
    encode_single, encode_batch = load_resnet_fallback()

print("Using encoder:", "e4e" if encode_single else "ResNet50")

# ----------------------------------------------------------
# HELPERS
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    return [os.path.join(root,f)
            for root,_,files in os.walk(folder)
            for f in files if f.lower().endswith(exts)]

def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError(f"Cannot read image {p}")
    return img

augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
])

def augment_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aug = augment(image=img_rgb)['image']
    return cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)

# ----------------------------------------------------------
# LOAD DATA
# ----------------------------------------------------------
imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)
print("Found REAL:", len(imgs_real), "FAKE:", len(imgs_fake))

# ----------------------------------------------------------
# ENCODING
# ----------------------------------------------------------
batch_size = 16
X_list, y_list = [], []

def process(paths, label, augment_real=False):
    buf = []
    for p in tqdm(paths):
        img = safe_imread(p)
        buf.append(img)
        if augment_real:
            buf.append(augment_image(img))
        if len(buf) >= batch_size:
            lat = encode_batch(buf)
            X_list.extend(lat)
            y_list.extend([label]*len(lat))
            buf = []
    if buf:
        lat = encode_batch(buf)
        X_list.extend(lat)
        y_list.extend([label]*len(lat))

process(imgs_real, 0, augment_real=True)
process(imgs_fake, 1, augment_real=False)

X = np.array(X_list)
y = np.array(y_list)

if X.ndim == 3:
    X = X.mean(axis=1)

np.save(os.path.join(output_dir,"latent_vectors.npy"), X)
np.save(os.path.join(output_dir,"labels.npy"), y)

# ----------------------------------------------------------
# STATISTICS (SAFE)
# ----------------------------------------------------------
def sample(arr, n=1500):
    if len(arr)<=n: return arr
    idx = np.random.choice(len(arr), n, replace=False)
    return arr[idx]

Xr, Xf = X[y==0], X[y==1]
Xr_s, Xf_s = sample(Xr), sample(Xf)

analysis = {
    "intra_real": float(np.mean(cdist(Xr_s,Xr_s))) if len(Xr_s)>1 else None,
    "intra_fake": float(np.mean(cdist(Xf_s,Xf_s))) if len(Xf_s)>1 else None,
    "inter": float(np.mean(cdist(Xr_s,Xf_s))) if len(Xr_s)>0 and len(Xf_s)>0 else None
}

json.dump(analysis, open(os.path.join(output_dir,"latent_analysis_extended.json"),"w"), indent=2)

# ----------------------------------------------------------
# STANDARDIZE + IPCA
# ----------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
ipca = IncrementalPCA(n_components=512, batch_size=1024)
for i in range(0,len(X_scaled),1024):
    ipca.partial_fit(X_scaled[i:i+1024])
X_red = ipca.transform(X_scaled)

joblib.dump(scaler, os.path.join(output_dir,"scaler.joblib"))
joblib.dump(ipca, os.path.join(output_dir,"incremental_pca_512.joblib"))

# ----------------------------------------------------------
# VISUALISATIONS
# ----------------------------------------------------------
X_3d = PCA(3).fit_transform(X_red)
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_3d[:,0],X_3d[:,1],X_3d[:,2],c=y,s=4,cmap="coolwarm")
plt.savefig(os.path.join(output_dir,"3D_features.png"))
plt.close()

# TSNE FIXED
tsne = TSNE(n_components=2, perplexity=40, max_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(sample(X_red,3000))
plt.scatter(X_tsne[:,0],X_tsne[:,1],s=4)
plt.savefig(os.path.join(output_dir,"tsne_2d.png"))
plt.close()

# ----------------------------------------------------------
# SMOTE + TRAIN
# ----------------------------------------------------------
X_res, y_res = SMOTE(random_state=42).fit_resample(X_red,y)
Xtr,Xte,ytr,yte = train_test_split(X_res,y_res,test_size=0.2,random_state=42)

svm = SVC(C=10,probability=True)
rf = RandomForestClassifier(n_estimators=500)
mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700)
xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, eval_metric="logloss")

for m in [svm,rf,mlp,xgb]:
    m.fit(Xtr,ytr)

voting = VotingClassifier(
    estimators=[("rf",rf),("mlp",mlp),("xgb",xgb)],
    voting="soft"
)
voting.fit(Xtr,ytr)

pred = voting.predict(Xte)
acc = accuracy_score(yte,pred)

cm = confusion_matrix(yte,pred)
sns.heatmap(cm,annot=True,fmt="d")
plt.savefig(os.path.join(output_dir,"confusion_matrix_final.png"))
plt.close()

joblib.dump(voting, os.path.join(output_dir,"model_voting.joblib"))

print("FINAL ACCURACY:", acc)
print("PIPELINE TERMINÉ ✅")



Device: cuda
Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt
e4e loaded successfully.
Using encoder: e4e
Found REAL: 790 FAKE: 3975


100%|██████████| 3975/3975 [07:48<00:00,  8.48it/s]
c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


FINAL ACCURACY: 0.9270440251572327
PIPELINE TERMINÉ ✅


In [2]:

import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
import torchvision.models as models
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from mpl_toolkits.mplot3d import Axes3D
import albumentations as A

from sklearn.manifold import TSNE
import umap.umap_ as umap
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

path_real = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\artifacts\frames\real"
path_fake = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\artifacts\frames\fake"

ckpt_e4e = os.path.join('encoder4editing', 'pretrained_models', 'e4e_ffhq_encode.pt')
output_dir = r"C:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\results_LSA_pipeline_optimisedddddd"
os.makedirs(output_dir, exist_ok=True)

use_gan_inversion = True

# ----------------------------------------------------------
# MODEL LOADING (e4e or ResNet fallback)
# ----------------------------------------------------------
def try_load_e4e(ckpt_path):
    try:
        from encoder4editing.models.psp import pSp
        ckpt = torch.load(ckpt_path, map_location='cpu', weights_only=True)
        opts = ckpt['opts']
        opts['checkpoint_path'] = ckpt_path
        opts = type('Options', (), opts)()
        net = pSp(opts).to(device).eval()

        tf_e4e = T.Compose([
            T.ToPILImage(),
            T.Resize((256,256)),
            T.ToTensor()
        ])

        def encode(img_bgr):
            img_rgb = img_bgr[:, :, ::-1]
            t = tf_e4e(img_rgb).unsqueeze(0).to(device)
            with torch.no_grad():
                out = net.encoder(t)
            return out.cpu().numpy().reshape(-1)

        def encode_batch(imgs_bgr_list):
            tensors = []
            for img_bgr in imgs_bgr_list:
                img_rgb = img_bgr[:, :, ::-1]
                tensors.append(tf_e4e(img_rgb))
            batch_t = torch.stack(tensors).to(device)
            with torch.no_grad():
                out = net.encoder(batch_t)
            return out.cpu().numpy()

        print("e4e loaded successfully.")
        return encode, encode_batch

    except Exception as e:
        print("Erreur chargement e4e :", e)
        return None, None

def load_resnet_fallback():
    model = models.resnet50(weights="DEFAULT")
    model.fc = torch.nn.Identity()
    model.to(device).eval()

    tf_res = T.Compose([
        T.ToPILImage(),
        T.Resize((224,224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])

    def encode(img_bgr):
        img_rgb = img_bgr[:, :, ::-1]
        t = tf_res(img_rgb).unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(t)
        return out.cpu().numpy().reshape(-1)

    def encode_batch(imgs_bgr_list):
        tensors = []
        for img_bgr in imgs_bgr_list:
            img_rgb = img_bgr[:, :, ::-1]
            tensors.append(tf_res(img_rgb))
        batch_t = torch.stack(tensors).to(device)
        with torch.no_grad():
            out = model(batch_t)
        return out.cpu().numpy()

    return encode, encode_batch

encode_single, encode_batch = None, None
if use_gan_inversion and os.path.exists(ckpt_e4e):
    encode_single, encode_batch = try_load_e4e(ckpt_e4e)

if encode_single is None:
    encode_single, encode_batch = load_resnet_fallback()

print("Using encoder:", "e4e" if encode_single else "ResNet50")

# ----------------------------------------------------------
# HELPERS
# ----------------------------------------------------------
def list_images(folder):
    exts = ('.jpg','.jpeg','.png','.bmp')
    return [os.path.join(root,f)
            for root,_,files in os.walk(folder)
            for f in files if f.lower().endswith(exts)]

def safe_imread(p):
    img = cv2.imread(p)
    if img is None:
        raise ValueError(f"Cannot read image {p}")
    return img

augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
])

def augment_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aug = augment(image=img_rgb)['image']
    return cv2.cvtColor(aug, cv2.COLOR_RGB2BGR)

# ----------------------------------------------------------
# LOAD DATA
# ----------------------------------------------------------
imgs_real = list_images(path_real)
imgs_fake = list_images(path_fake)
print("Found REAL:", len(imgs_real), "FAKE:", len(imgs_fake))

# ----------------------------------------------------------
# ENCODING
# ----------------------------------------------------------
batch_size = 16
X_list, y_list = [], []

def process(paths, label, augment_real=False):
    buf = []
    for p in tqdm(paths):
        img = safe_imread(p)
        buf.append(img)
        if augment_real:
            buf.append(augment_image(img))
        if len(buf) >= batch_size:
            lat = encode_batch(buf)
            X_list.extend(lat)
            y_list.extend([label]*len(lat))
            buf = []
    if buf:
        lat = encode_batch(buf)
        X_list.extend(lat)
        y_list.extend([label]*len(lat))

process(imgs_real, 0, augment_real=True)
process(imgs_fake, 1, augment_real=False)

X = np.array(X_list)
y = np.array(y_list)

if X.ndim == 3:
    X = X.mean(axis=1)

np.save(os.path.join(output_dir,"latent_vectors.npy"), X)
np.save(os.path.join(output_dir,"labels.npy"), y)

# ----------------------------------------------------------
# STATISTICS (SAFE)
# ----------------------------------------------------------
def sample(arr, n=1500):
    if len(arr)<=n: return arr
    idx = np.random.choice(len(arr), n, replace=False)
    return arr[idx]

Xr, Xf = X[y==0], X[y==1]
Xr_s, Xf_s = sample(Xr), sample(Xf)

analysis = {
    "intra_real": float(np.mean(cdist(Xr_s,Xr_s))) if len(Xr_s)>1 else None,
    "intra_fake": float(np.mean(cdist(Xf_s,Xf_s))) if len(Xf_s)>1 else None,
    "inter": float(np.mean(cdist(Xr_s,Xf_s))) if len(Xr_s)>0 and len(Xf_s)>0 else None
}

json.dump(analysis, open(os.path.join(output_dir,"latent_analysis_extended.json"),"w"), indent=2)

# ----------------------------------------------------------
# STANDARDIZE + IPCA
# ----------------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
ipca = IncrementalPCA(n_components=512, batch_size=1024)
for i in range(0,len(X_scaled),1024):
    ipca.partial_fit(X_scaled[i:i+1024])
X_red = ipca.transform(X_scaled)

joblib.dump(scaler, os.path.join(output_dir,"scaler.joblib"))
joblib.dump(ipca, os.path.join(output_dir,"incremental_pca_512.joblib"))

# ----------------------------------------------------------
# VISUALISATIONS
# ----------------------------------------------------------
X_3d = PCA(3).fit_transform(X_red)
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_3d[:,0],X_3d[:,1],X_3d[:,2],c=y,s=4,cmap="coolwarm")
plt.savefig(os.path.join(output_dir,"3D_features.png"))
plt.close()

# TSNE FIXED
tsne = TSNE(n_components=2, perplexity=40, max_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(sample(X_red,3000))
plt.scatter(X_tsne[:,0],X_tsne[:,1],s=4)
plt.savefig(os.path.join(output_dir,"tsne_2d.png"))
plt.close()

# ----------------------------------------------------------
# SMOTE + TRAIN
# ----------------------------------------------------------
X_res, y_res = SMOTE(random_state=42).fit_resample(X_red,y)
Xtr,Xte,ytr,yte = train_test_split(X_res,y_res,test_size=0.2,random_state=42)

svm = SVC(C=10,probability=True)
rf = RandomForestClassifier(n_estimators=500)
mlp = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=700)
xgb = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=6, eval_metric="logloss")

for m in [svm,rf,mlp,xgb]:
    m.fit(Xtr,ytr)

voting = VotingClassifier(
    estimators=[("rf",rf),("mlp",mlp),("xgb",xgb)],
    voting="soft"
)
voting.fit(Xtr,ytr)

pred = voting.predict(Xte)
acc = accuracy_score(yte,pred)

cm = confusion_matrix(yte,pred)
sns.heatmap(cm,annot=True,fmt="d")
plt.savefig(os.path.join(output_dir,"confusion_matrix_final.png"))
plt.close()

joblib.dump(voting, os.path.join(output_dir,"model_voting.joblib"))

print("FINAL ACCURACY:", acc)
print("PIPELINE TERMINÉ ✅")


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


c:\Users\EliteLaptop\miniconda3\envs\torch_gpu\lib\site-packages\torch\utils\cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Loading e4e over the pSp framework from checkpoint: encoder4editing\pretrained_models\e4e_ffhq_encode.pt


c:\Users\EliteLaptop\Desktop\kawtar\GAN_inversion\encoder4editing\models\psp.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path

e4e loaded successfully.
Using encoder: e4e
Found REAL: 12361 FAKE: 62877


 69%|██████▉   | 43439/62877 [59:18<26:32, 12.21it/s]  


KeyboardInterrupt: 